# Genetic Algorithm Merit Matrix Optimizer

## Цель оптимизации

Найти оптимальную merit increase matrix (матрицу повышений зарплат), которая:
1. Соответствует бюджету — итоговые затраты попадают в диапазон [target - lower_tol%, target + upper_tol%]
2. Устойчива к вариациям — корректно работает при разных распределениях performance ratings от менеджеров
3. Соблюдает структурные ограничения — монотонность, минимальные/максимальные шаги, anchor cells
4. Дифференцирует performance — заметная разница между high/low performers

---

## Ключевые особенности

Гибкая конфигурация:
- Поддержка произвольного количества ratings (2+) и CR bins (2+)
- Custom target distributions с возможностью hard zeros (исключение рейтингов из Monte Carlo)
- Custom CR bin edges для нестандартной сегментации
- Asymmetric budget tolerance — разные пороги для under/overspend

Продвинутая валидация:
- Strict zero enforcement — принудительное обнуление выбранных ячеек матрицы
- Anchor cells — фиксация угловых ячеек (best/worst performers) на точных значениях
- Auto-adaptive constraints на основе размера merit pool

Оптимизация производительности:
- Векторизованный fitness evaluation через pre-stacked scenarios
- Эффективное построение salary matrix с groupby (O(N) вместо O(N×I×J))
- JIT-компиляция критичных функций для больших датасетов

---

## Методология

Genetic Algorithm:
1. Initialization — создание начальной популяции структурированных матриц
2. Evolution — tournament selection, crossover, mutation в течение N поколений
3. Fitness Evaluation — быстрая оценка на subset сценариев (quick eval)
4. Full Evaluation — детальная оценка топ-кандидатов на полном наборе сценариев
5. Local Refinement — оптимизация лучших решений через L-BFGS-B

Monte Carlo Simulation:
- Генерация тысяч сценариев распределения ratings через Dirichlet perturbation
- Stress testing — включение экстремальных распределений (inflated, harsh, forced curve)
- Budget validation — проверка попадания в диапазон для каждого сценария

Constraint System:
- Cell bounds — глобальные min/max для любой ячейки
- Monotonicity — ratings растут слева направо, CR уменьшается сверху вниз
- Step sizes — минимальные/максимальные изменения между соседними ячейками
- Multipliers — ratio-требования между разными ratings
- Fixed cells — strict zeros и anchor points

## Setup and Imports

### Библиотеки для оптимизации

**Научные вычисления:**
- `numpy` — векторизованные операции, массивы для матриц и сценариев
- `pandas` — обработка employee data, группировка для salary matrices
- `scipy.optimize.minimize` — L-BFGS-B для local refinement
- `scipy.stats.norm` — генерация bell curve distributions, Dirichlet perturbation

**Структуры данных:**
- `dataclasses` — type-safe структуры для MatrixCandidate, PolicyGuidance
- `typing` — аннотации типов для повышения читаемости кода

**Утилиты:**
- `json` — экспорт конфигурации
- `warnings` — подавление RuntimeWarnings при работе с NaN/inf
- `os` — работа с файловой системой для экспорта

**Design Note:** код использует функциональный стиль и чистые функции для репродуцируемости

In [ ]:
import numpy as np
import pandas as pd
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
import json
import warnings
from scipy.optimize import minimize
from scipy.stats import norm
import os

warnings.filterwarnings('ignore', category=RuntimeWarning, message='invalid value encountered')

## User Configuration Section

КРИТИЧЕСКАЯ СЕКЦИЯ — все параметры оптимизации задаются здесь.

---

### 1. Budget Configuration

**MERIT_POOL_VALUE:**
- Целевой средний процент повышения по компании (например, 0.08 = 8%)
- Используется для расчета total merit budget = MERIT_POOL_VALUE × total_payroll

**BUDGET_TOLERANCE_LOWER / UPPER (Asymmetric):**
- Допустимые отклонения от target budget
- Lower: сколько можно НЕ ПОТРАТИТЬ (например, 0.05 = допустимо -5% от бюджета)
- Upper: сколько можно ПЕРЕРАСХОДОВАТЬ (например, 0.015 = допустимо +1.5% от бюджета)
- Asymmetry отражает бизнес-реальность: underspend часто более приемлем, чем overspend

---

### 2. Matrix Structure

**NUM_RATINGS:**
- Количество уровней performance ratings (минимум 2)
- Типичные значения: 3 (Below/Meets/Exceeds), 5 (стандарт), 7 (детальная система)

**NUM_CR_BINS:**
- Количество групп по Compa-Ratio (минимум 2)
- Игнорируется, если задан CUSTOM_CR_BINS
- Больше bins = более детальная дифференциация, но выше риск overfitting

**CR_BIN_FIRST / CR_BIN_LAST:**
- Границы для auto-generation bins
- Например: 0.75 to 1.25 создает bins вокруг рыночного уровня (CR=1.0)

---

### 3. Target Distribution

**CUSTOM_TARGET_DISTRIBUTION:**
- `None` — автоматическая bell curve генерация
- `Dict[int, float]` — точное распределение по ratings
- Должно суммироваться в 1.0
- Может содержать zeros для исключения ratings

**HARD_ZERO_RATINGS:**
- `False` — zero-probability ratings могут появляться в Monte Carlo с epsilon-вероятностью
- `True` — zero-probability ratings полностью исключаются из всех сценариев
- Применяется только если CUSTOM_TARGET_DISTRIBUTION содержит zeros

---

### 4. Monte Carlo Settings

**SCENARIO_CONCENTRATION:**
- Контролирует variance в Dirichlet perturbation
- Выше значение → сценарии ближе к target distribution (меньше variance)
- Ниже значение → больше разброс (stress testing)
- Типичное: ~166 (5%/0.03) для ±10% отклонений от target
- Для peaky distributions автоматически увеличивается

---

### 5. Custom CR Bins

**CUSTOM_CR_BINS:**
- `None` — автоматическая генерация на основе NUM_CR_BINS
- `List[float]` — явное задание границ (начинается с 0.0, заканчивается np.inf)
- Минимум 3 edges (даёт ≥2 bins)
- Пример: [0.0, 0.80, 0.90, 1.10, 1.20, np.inf] → 5 bins с custom ranges

---

### 6. Strict Zero Enforcement

**FORCE_ZERO_CELLS:**
- Принудительное обнуление выбранных ячеек матрицы
- Формат: {cr_bin_index: [list of rating indices]}
- Пример: {4: [0, 1]} — CR bin 4, ratings 1–2 всегда 0%
- `None` или `{}` — отключено, используется только ANCHOR_CELL контроль

---

### 7. Anchor Cell Controls

**ANCHOR_CELL_MAX:**
- Точное значение merit % для best performers (CR bin 0, highest rating)
- `None` — свободно, `float` — фиксируется
- Пример: 0.10 = 10% для top performers

**ANCHOR_CELL_MIN:**
- Точное значение merit % для worst performers (highest CR bin, lowest rating)
- `None` — свободно, `0.0` — zero merit, `float` — другое значение
- Anchor cells PINNED на протяжении всей оптимизации

---

### 8. Constraint Configuration

**CONSTRAINT_CONFIG:**
- Auto-adaptive constraints на основе merit pool и matrix size
- `None` значения → автоматический расчет
- Можно вручную override для экспериментов

**Key constraints:**
- `cell_min/max` — bounds для любой ячейки
- `min/max_step_rating` — шаги между ratings
- `min/max_step_cr` — шаги между CR bins
- `rating_multiplier` — ratio-требования (например, Rating 5 ≥ 1.4× Rating 1)

---

### 9. Genetic Algorithm Parameters

**GA_CONFIG:**
- `population_size` — размер популяции (больше = лучше exploration)
- `num_generations` — максимум итераций (может сойтись ранее)
- `elite_size` — сколько лучших сохраняются без изменений
- `tournament_size` — размер турнира для selection
- `mutation_rate` — вероятность мутации ячейки (0.25 = 25%)
- `crossover_rate` — вероятность crossover vs клонирования
- `convergence_patience` — остановка при стагнации fitness

---

### 10. Fitness Weights

**FITNESS_WEIGHTS:**
- `budget` — вес budget score (попадание в tolerance band)
- `constraints` — вес constraint satisfaction (monotonicity, steps, etc.)
- Суммарно = 1.0

**DIFF_WEIGHT:**
- Доп. вес для видимой дифференциации
- 0.0 = отключено
- 0.10–0.15 = лёгкий nudge
- 0.30+ = сильное влияние
- Поощряет steeper slopes между ratings для visibility

---

### 11. Evaluation Strategy

**EVAL_CONFIG:**
- `quick_eval_scenarios` — сценариев для быстрой GA оценки (2000–5000)
- `full_eval_scenarios` — сценариев для финальной оценки (10000+)
- `num_top_candidates` — сколько лучших идут в full evaluation

---

### 12. Data & Seeds

**DATA_FILE:**
- Путь к Excel с employee data (`base_salary`, `CR`)
- `None` — сгенерировать synthetic data

**SEED_*:**
- Random seeds для репродуцируемости
- Разные seeds для population, scenarios, GA → независимость


In [ ]:
# ======================
# USER CONFIGURATION
# ======================

# Merit pool configuration
MERIT_POOL_VALUE = 13/100

# Budget tolerance (asymmetric)
# Lower tolerance: how much UNDER budget is acceptable (underspending)
# Upper tolerance: how much OVER budget is acceptable (overspending)
# Example: LOWER=0.05, UPPER=0.015 means -5% to +1.5% from target is acceptable
BUDGET_TOLERANCE_LOWER = 0.05  # 6% underspend allowed
BUDGET_TOLERANCE_UPPER = 0.05 # 1.5% overspend allowed

# Rating structure
NUM_RATINGS = 5  # Can be any integer >= 2
NUM_CR_BINS = 5   # Can be any integer >= 2 (ignored if CUSTOM_CR_BINS is set)
CR_BIN_FIRST = 0.75
CR_BIN_LAST = 1.25

# Custom target rating distribution (Optional)
# Set to None for automatic bell curve generation based on NUM_RATINGS
# Set to dict to specify custom distribution (must sum to 1.0, can include zeros)
# Example: CUSTOM_TARGET_DISTRIBUTION = {1: 0.05, 2: 0.15, 3: 0.40, 4: 0.30, 5: 0.10}
# Example with zeros: CUSTOM_TARGET_DISTRIBUTION = {1: 0.0, 2: 0.1, 3: 0.6, 4: 0.2, 5: 0.1}
# Keys must match rating scale (1 to NUM_RATINGS)
CUSTOM_TARGET_DISTRIBUTION = {1: 0.10, 2: 0.15, 3: 0.50, 4: 0.15, 5: 0.10}

# Hard zero ratings (applies only when custom distribution contains zeros)
# False: Zero-probability ratings can still appear in Monte Carlo with tiny epsilon mass
# True: Zero-probability ratings are completely excluded from Monte Carlo scenarios
# Example: If rating 1 has 0% target and HARD_ZERO_RATINGS=True, rating 1 never appears
HARD_ZERO_RATINGS = False

# Monte Carlo scenario variance
# Higher values = scenarios stay closer to target distribution
# Lower values = more variance in rating distributions across scenarios
# Default ~66.67 keeps scenarios within ~10% of target percentages
SCENARIO_CONCENTRATION = 5 / 0.03

# Custom CR bin edges (Optional)
# Set to None for automatic bin generation based on NUM_CR_BINS, CR_BIN_FIRST, CR_BIN_LAST
# Set to list of edges to use custom bins (must start with 0.0 and end with np.inf)
# Example: CUSTOM_CR_BINS = [0.0, 0.80, 0.90, 1.00, 1.10, 1.20, np.inf]
# When custom bins are used, NUM_CR_BINS is automatically set to len(CUSTOM_CR_BINS) - 1
# REQUIRES at least 3 edges (defines at least 2 bins)
CUSTOM_CR_BINS = [0.0, 0.80, 0.90, 1.10, 1.20, np.inf]

# STRICT ZERO ENFORCEMENT (Optional - for multiple cells)
# Force zero merit for multiple cells beyond just the worst performer corner
# Format: {cr_bin_index: [list of rating indices to zero]}
# Example: {4: [0, 1], 3: [0]} means CR bin 4 ratings 1-2 get 0%, CR bin 3 rating 1 gets 0%
# 
# Set to None or {} to disable and use ANCHOR_CELL_MIN instead for corner control
# Use this when you need zeros in MULTIPLE cells, not just the worst corner
FORCE_ZERO_CELLS = None  # Disabled - using ANCHOR_CELL controls instead


# FORCE_ZERO_CELLS = {
#     NUM_CR_BINS - 1: [0],  # Highest CR bin (worst positioned), lowest rating
#     NUM_CR_BINS - 2: [0],  # Second highest CR bin, lowest rating
# }


# ANCHOR CELL CONTROLS (Recommended for corner control)
# Set exact values for corner cells of the matrix
# anchor_max: Exact merit % for best performers (lowest CR bin, highest rating)
# anchor_min: Exact merit % for worst performers (highest CR bin, lowest rating)
# When set to specific values, these cells are PINNED throughout optimization
# When set to None, optimizer is free to explore within normal constraints
ANCHOR_CELL_MAX = None  # e.g., 0.10 to pin best performers at exactly 10%
ANCHOR_CELL_MIN = None   # e.g., 0.01 to pin worst performers at exactly 1%, or 0.0 for zero

# Data source
DATA_FILE = 'Misha.xlsx'  # Set to None for synthetic data

# Random seeds
SEED_BASE_POPULATION = 42
SEED_SCENARIOS = 2025
SEED_GA = 89

# Output directory
OUTPUT_DIR = "artifacts"

# Constraint configuration
CONSTRAINT_CONFIG = {
    'cell_min': 0,
    'cell_max': None,
    'min_step_rating': None,
    'step_max_rating': None,
    'min_step_cr': None,
    'step_max_cr': None,
    'rating_multiplier': None,
}

# Genetic Algorithm Parameters
GA_CONFIG = {
    'population_size': 1000,
    'num_generations': 500,
    'elite_size': 100,
    'tournament_size': 5,
    'mutation_rate': 0.25,
    'crossover_rate': 0.7,
    'convergence_patience': 50,
    'convergence_threshold': 1e-5,
}

# Fitness weights
FITNESS_WEIGHTS = {
    'budget': 0.70,
    'constraints': 0.30
}

# Extra weight to reward visible rating differentiation (0..1)
# Set to 0.10–0.15 for a noticeable nudge without blowing up budget risk.
DIFF_WEIGHT = 0.30

# Evaluation strategy
EVAL_CONFIG = {
    'quick_eval_scenarios': 2000,
    'full_eval_scenarios': 10000,
    'num_top_candidates': 20,
}

## Auto-Generated Configuration

**Автоматическая генерация параметров на основе пользовательских настроек.**

---

### 1. Генерация целевого распределения (Target Distribution Generation)

**generate_target_distribution():**
- Создаёт нормальное распределение (bell curve) для `NUM_RATINGS`
- Центрируется на среднем рейтинге: `loc = (NUM_RATINGS - 1) / 2`
- `scale` адаптируется к количеству рейтингов: `scale = NUM_RATINGS / 4`
- Используется, если `CUSTOM_TARGET_DISTRIBUTION = None`

**Пример для 5 ratings:**
- Rating 1: ~10% (хвосты)
- Rating 2: ~20%
- Rating 3: ~40% (центр)
- Rating 4: ~20%
- Rating 5: ~10% (хвосты)

---

### 2. Валидация пользовательского распределения (Custom Distribution Validation)

**validate_custom_target_distribution():**
- ДОЛЖНА быть вызвана перед использованием custom distribution
- Проверяет:
  - Тип = `dict`
  - Ключи соответствуют диапазону рейтингов (1…NUM_RATINGS)
  - Значения числовые, ≥0, ≤1.0
  - Сумма = 1.0 (с допуском 1e-6)
- Подход “fail fast” — при ошибке сразу выбрасывает `ValueError` (быстрое завершение при некорректных данных)

---

### 3. Вычисление параметров Дирихле (Dirichlet Alpha Computation)

**compute_dirichlet_alphas():**
- Преобразует вероятности целевого распределения в параметры Дирихле (`alpha`)
- Обработка нулей: `epsilon = 1e-12`, чтобы все `alpha` были строго > 0
- Масштабирование на основе энтропии (entropy-based scaling):
  - Узкие/пиковые распределения (peaky distributions) → увеличиваем `concentration`, чтобы снизить разброс (variance)
- Формула:  
  `alpha = (probs / sum(probs)) × concentration × (1 + peakiness_factor)`

---

### 4. Семплирование из Дирихле (Dirichlet Sampling)

**dirichlet_sample_from_target():**
- Генерирует “возмущённое” (perturbed) распределение вокруг target
- **Hard zeros mode (жёсткие нули):** если `HARD_ZERO_RATINGS=True`:
  - Семплируется только из рейтингов с >0
  - Полное распределение восстанавливается с нулями
- **Soft zeros mode (мягкие нули):** применяется `epsilon` ко всем ratings

---

### 5. Маскирование нулей в стресс-сценариях (Zero Masking for Stress Scenarios)

**mask_zeros_in_distribution():**
- Применяет zero constraints к стресс-сценариям
- Обнуляет рейтинги, которые имеют 0% в target
- Нормализует оставшиеся рейтинги до sum=1.0
- Важно для консистентности при `HARD_ZERO_RATINGS=True`

---

### 6. Генерация CR-бинов (CR Bin Generation)

**generate_cr_bin_edges():**
- Автоматически генерирует равномерно распределённые bins
- Формат: `[0.0, ..., ..., np.inf]`
- Промежуточные границы: `linspace(CR_BIN_FIRST, CR_BIN_LAST, NUM_CR_BINS - 1)`

**validate_custom_cr_bins():**
- Проверяет корректность пользовательских границ
- Требования:
  - Минимум 3 границы (даёт ≥2 bins)
  - Первая граница = `0.0` (ТОЧНО, для совместимости с `pd.cut`)
  - Последняя граница = `np.inf`
  - Строго возрастающие значения
  - Без отрицательных значений

---

### 7. Генерация стресс-сценариев (Stress Scenario Generation)

**generate_stress_scenarios():**
- Создаёт экстремальные распределения для устойчивости (robust testing)
- Типы:
  - `inflated` — смещение к высоким рейтингам (grade inflation — “раздувание оценок”)
  - `harsh` — смещение к низким рейтингам (строгие менеджеры)
  - `forced_curve` — искусственная bell curve независимо от target
  - `top_heavy` — концентрация в верхней половине
- **honor_zeros:** если True, применяет маскирование нулей (zero masking) ко всем stress scenarios

---

### 8. Обработка фиксированных ячеек (Fixed Cell Processing)

**process_force_zero_cells():**
- Конвертирует `FORCE_ZERO_CELLS` в boolean mask
- `Mask[i, j] = True` → ячейка `[i, j]` ОБЯЗАТЕЛЬНО = 0
- Валидация индексов, предупреждения (warnings) при выходе за диапазон

**get_anchor_cells():**
- Извлекает координаты и значения anchor cells
- Возвращает словарь: `{'max': (i, j, value), 'min': (i, j, value)}`
- Используется для жёсткого применения (enforcement) и отчётности (reporting)

**enforce_anchor_cells() / enforce_strict_zeros():**
- Принудительно применяют фиксированные значения к матрице
- Вызываются после КАЖДОЙ модификации (mutation, crossover, repair)

---

### 9. Объекты конфигурации (Configuration Objects)

После валидации создаются глобальные объекты:
- `TARGET_RATING_DISTRIBUTION` — финализированное распределение
- `CR_BIN_EDGES` — итоговые границы CR-бинов
- `NUM_CR_BINS` — обновляется при использовании CUSTOM_CR_BINS
- `CONSTRAINTS` — автоматически сгенерированные ограничения
- `STRESS_SCENARIOS` — предвычисленные стресс-сценарии
- `STRICT_ZERO_MASK` — boolean mask для принудительных нулей

---

### Принципы проектирования (Design Philosophy)

- **Проверяй раннее, завершай быстро (validate early, fail fast):**  
  Ошибки лучше ловить на входе, чем отлавливать последствия позже.

- **Автоматически адаптируйся к входным данным (auto-adapt to inputs):**  
  Логика подстраивается под конфигурацию пользователя, вместо жёстких значений.

- **Явное лучше, чем неявное (explicit over implicit):**  
  Все ключевые решения должны быть заданы напрямую, без скрытой логики.


In [ ]:
# ======================
# AUTO-GENERATED CONFIG
# ======================

RATING_ORDER = list(range(1, NUM_RATINGS + 1))


def generate_target_distribution(num_ratings: int) -> Dict[int, float]:
    """Auto-generate target distribution based on NUM_RATINGS (bell curve pattern)"""
    
    ratings = np.arange(num_ratings)
    
    # Use bell curve centered on middle rating
    # Scale parameter adapts to number of ratings for natural spread
    probs = norm.pdf(ratings, loc=(num_ratings-1)/2, scale=num_ratings/4)
    probs = probs / probs.sum()
    
    # Convert to dict with 1-indexed ratings
    return {i+1: float(p) for i, p in enumerate(probs)}


def validate_custom_target_distribution(dist: Optional[Dict], num_ratings: int) -> None:
    """Validate custom target distribution - MUST be called before using distribution"""
    if dist is None:
        return
    
    errors = []
    
    # Check it's a dict
    if not isinstance(dist, dict):
        errors.append("CUSTOM_TARGET_DISTRIBUTION must be a dictionary")
        raise ValueError("Custom target distribution validation failed:\n  " + "\n  ".join(errors))
    
    # Check keys match rating order
    expected_keys = set(range(1, num_ratings + 1))
    actual_keys = set(dist.keys())
    
    if actual_keys != expected_keys:
        errors.append(f"CUSTOM_TARGET_DISTRIBUTION keys must be {expected_keys}, got {actual_keys}")
    
    # Check all values are numeric and non-negative
    for rating, prob in dist.items():
        if not isinstance(prob, (int, float)):
            errors.append(f"Rating {rating} probability must be numeric, got {type(prob)}")
        elif prob < 0:
            errors.append(f"Rating {rating} probability must be non-negative, got {prob}")
        elif prob > 1:
            errors.append(f"Rating {rating} probability must be <= 1.0, got {prob}")
    
    # Check probabilities sum to 1.0 (with tolerance)
    if not errors:  # Only check sum if individual values are valid
        total = sum(dist.values())
        if abs(total - 1.0) > 1e-6:
            errors.append(f"CUSTOM_TARGET_DISTRIBUTION must sum to 1.0, got {total:.8f}")
    
    if errors:
        raise ValueError("Custom target distribution validation failed:\n  " + "\n  ".join(errors))


def compute_dirichlet_alphas(base_probs: np.ndarray, concentration: float) -> np.ndarray:
    """
    Compute Dirichlet alpha parameters from target probabilities, handling zeros.
    
    Args:
        base_probs: Target probability distribution (may contain zeros)
        concentration: Controls variance around target (higher = less variance)
    
    Returns:
        Alpha parameters suitable for np.random.dirichlet (all strictly positive)
    """
    # Ensure strictly positive alphas (Dirichlet requires alpha > 0)
    eps = 1e-12
    alphas = np.clip(base_probs, eps, None)
    
    # Calculate entropy to detect peaky distributions
    entropy = -np.sum(base_probs * np.log(np.clip(base_probs, eps, 1.0)))
    max_entropy = np.log(len(base_probs))
    
    # Peakiness factor: 0 = uniform, 1 = extremely peaked
    peakiness_factor = (max_entropy - entropy) / max_entropy if max_entropy > 0 else 0.0
    
    # Scale up concentration for peaky targets to reduce variance
    scale = 1.0 + 1.0 * peakiness_factor
    
    # Renormalize to maintain total concentration "mass"
    alphas = alphas / alphas.sum() * concentration * scale
    
    return alphas


def dirichlet_sample_from_target(base_probs: np.ndarray, concentration: float, rng) -> np.ndarray:
    """
    Sample from Dirichlet distribution, respecting hard zeros if enabled.
    
    Args:
        base_probs: Target probability distribution (may contain zeros)
        concentration: Controls variance around target
        rng: Random number generator
    
    Returns:
        Perturbed probability distribution
    """
    if HARD_ZERO_RATINGS and np.any(base_probs == 0):
        # Hard zeros: exclude zero-probability ratings entirely
        nz_idx = np.flatnonzero(base_probs > 0)
        sub_probs = base_probs[nz_idx] / base_probs[nz_idx].sum()
        sub_alphas = compute_dirichlet_alphas(sub_probs, concentration)
        sub_draw = rng.dirichlet(sub_alphas)
        
        # Reconstruct full distribution with zeros
        draw = np.zeros_like(base_probs)
        draw[nz_idx] = sub_draw
        return draw
    else:
        # Soft zeros: use epsilon for zero probabilities
        alphas = compute_dirichlet_alphas(base_probs, concentration)
        return rng.dirichlet(alphas)


def mask_zeros_in_distribution(dist: Dict[int, float], target: Dict[int, float]) -> Dict[int, float]:
    """
    Force zero probabilities in dist where target has zeros, then renormalize.
    
    Args:
        dist: Distribution to mask
        target: Target distribution (determines which ratings to zero)
    
    Returns:
        Masked and renormalized distribution
    """
    masked = {k: (0.0 if target[k] == 0 else v) for k, v in dist.items()}
    total = sum(masked.values())
    
    if total > 0:
        return {k: v / total for k, v in masked.items()}
    else:
        # Degenerate case: all ratings were zero, distribute uniformly among non-zero targets
        non_zero_count = sum(1 for v in target.values() if v > 0)
        if non_zero_count > 0:
            return {k: (1.0 / non_zero_count if target[k] > 0 else 0.0) for k in dist.keys()}
        else:
            # All targets are zero - shouldn't happen with valid config
            return {k: 1.0 / len(dist) for k in dist.keys()}


def process_force_zero_cells(force_zero_config, num_cr_bins: int, num_ratings: int) -> np.ndarray:
    """Convert FORCE_ZERO_CELLS config to boolean mask"""
    mask = np.zeros((num_cr_bins, num_ratings), dtype=bool)
    
    if force_zero_config is None or not force_zero_config:
        return mask
    
    for cr_bin, rating_indices in force_zero_config.items():
        if cr_bin < 0 or cr_bin >= num_cr_bins:
            print(f"Warning: CR bin {cr_bin} out of range [0, {num_cr_bins-1}], skipping")
            continue
        for rating_idx in rating_indices:
            if rating_idx < 0 or rating_idx >= num_ratings:
                print(f"Warning: Rating index {rating_idx} out of range [0, {num_ratings-1}], skipping")
                continue
            mask[cr_bin, rating_idx] = True
    
    return mask


def get_anchor_cells() -> Dict[str, Tuple[int, int, float]]:
    """Get anchor cell coordinates and values if specified"""
    anchors = {}
    
    if ANCHOR_CELL_MAX is not None:
        # Best performers: CR bin 0 (lowest CR), highest rating
        anchors['max'] = (0, NUM_RATINGS - 1, ANCHOR_CELL_MAX)
    
    if ANCHOR_CELL_MIN is not None:
        # Worst performers: highest CR bin, rating 0 (lowest rating)
        anchors['min'] = (NUM_CR_BINS - 1, 0, ANCHOR_CELL_MIN)
    
    return anchors


def enforce_anchor_cells(matrix: np.ndarray) -> np.ndarray:
    """Enforce exact values for anchor cells"""
    matrix = matrix.copy()
    anchors = get_anchor_cells()
    
    for anchor_type, (i, j, value) in anchors.items():
        matrix[i, j] = value
    
    return matrix


def generate_cr_bin_edges(num_bins: int, first: float, last: float) -> List[float]:
    """Generate CR bin edges"""
    if num_bins < 2:
        raise ValueError("NUM_CR_BINS must be at least 2")
    intermediate = np.linspace(first, last, num_bins - 1)
    return [0.0] + list(intermediate) + [np.inf]


def validate_custom_cr_bins(bins) -> None:
    """Validate custom CR bin edges - MUST be called before using bins"""
    if bins is None:
        return
    
    errors = []
    
    # Check it's a list or array
    if not isinstance(bins, (list, tuple, np.ndarray)):
        errors.append("CUSTOM_CR_BINS must be a list, tuple, or numpy array")
        raise ValueError("Custom CR bins validation failed:\n  " + "\n  ".join(errors))
    
    bins_list = list(bins)
    
    # REQUIRE AT LEAST 3 EDGES (i.e., >= 2 bins)
    if len(bins_list) < 3:
        errors.append("CUSTOM_CR_BINS must have at least 3 edges (defines >= 2 bins)")
    
    # First edge must be EXACTLY 0.0 (pd.cut requires this for include_lowest=True)
    if len(bins_list) > 0 and bins_list[0] != 0.0:
        errors.append(f"CUSTOM_CR_BINS must start with exactly 0.0, got {bins_list[0]}")
    
    # Last edge must be np.inf
    if len(bins_list) > 0 and not np.isinf(bins_list[-1]):
        errors.append(f"CUSTOM_CR_BINS must end with np.inf, got {bins_list[-1]}")
    
    # Must be strictly increasing
    for i in range(len(bins_list) - 1):
        if not (bins_list[i] < bins_list[i+1]):
            errors.append(
                f"CUSTOM_CR_BINS must be strictly increasing, but "
                f"bins[{i}]={bins_list[i]} >= bins[{i+1}]={bins_list[i+1]}"
            )
    
    # All finite values must be non-negative
    for i, edge in enumerate(bins_list[:-1]):  # Exclude last (inf)
        if edge < 0:
            errors.append(f"CUSTOM_CR_BINS edges must be non-negative, but bins[{i}]={edge}")
    
    if errors:
        raise ValueError("Custom CR bins validation failed:\n  " + "\n  ".join(errors))

## Auto-Adaptive Constraint Configuration

КРИТИЧЕСКАЯ ФУНКЦИЯ для создания ограничений, которые одновременно реализуемы (feasible) и содержательно обоснованы (meaningful).

---

### Design Philosophy (Принципиальный подход)

Старый подход (проблемный):
- Ограничения строились на теоретическом диапазоне (0% до max_possible%)
- Требовали слишком широкие границы → решения становились нереализуемыми
- Пример: при merit pool 8% требовали span 0–24% → физически невозможно

Новый подход (adaptive):
- Ограничения строятся на реальном размере MERIT_POOL, а не на теоретическом max
- Ограничения направляют к хорошим решениям, а не блокируют их
- Баланс между видимостью различий (visibility) и выполнимостью (feasibility)

---

### 1. Границы ячеек (Cell Bounds, глобальные)

cell_min:
- По умолчанию: 0.0 (разрешает 0% для худших исполнителей)
- Можно переопределить для гарантированного минимума

cell_max:
- Адаптивный множитель, зависящий от размера merit pool
- Логика:
  - Малый pool (≤3%) → multiplier 4.0× (сильная дифференциация)
  - Средний pool (5–8%) → 3.0×
  - Большой pool (12–20%) → 2.0×
- Это потолок (ceiling), а не target → оптимизатор может использовать меньше

---

### 2. Горизонтальные шаги по рейтингам (Rating Step Sizes)

Принцип: рассчитывать от размера merit pool, а не от теоретического span

min_step_rating:
- Базово: 15% от merit pool на 1 шаг
- Пример: 8% pool → минимум 1.2% между соседними рейтингами
- Корректировки:
  - 7+ ratings → уменьшить на 25%
  - Минимум (floor): 0.5%
  - Максимум (ceiling): 3.0%

step_max_rating:
- 3.5× от минимального шага
- Жёсткий лимит: 8%

Обоснование:
- Гарантирует видимую дифференциацию performance
- Не перегружает оптимизатор
- Масштабируется с размером pool и количеством rating уровней

---

### 3. Вертикальные шаги по CR (CR Step Sizes)

Принцип: CR — вторичный фактор, поэтому шаги меньше

min_step_cr:
- 50% от min_step_rating
- Корректировки:
  - 7+ bins → уменьшить на 25%
  - Минимум: 0.3%
  - Максимум: 2.0%

step_max_cr:
- 3.0× от минимального шага
- Жёсткий лимит: 5%

---

### 4. Ограничения-множители (Multiplier Constraints / Ratios)

Цель: обеспечить логичный порядок между рейтингами

rating_multiplier:
- Формат: {low_rating_idx: (high_rating_idx, minimum_ratio)}
- Смягчение требований:
  - Старое правило: Top ≥1.8× Bottom → слишком жёстко
  - Новое правило: Top ≥1.4× Bottom → реализуемо
- Если ≥4 ratings: второй сверху ≥1.25× Bottom

Примечание: Multipliers обеспечивают порядок, а не экстремальную разницу

---

### 5. Диагностика реализуемости (Feasibility Diagnostics)

Автоматические проверки:

1. Range utilization (использование диапазона):
   - Сколько % доступного span занимают constraints?
   - >90% → Warning (слишком жёсткие ограничения)

2. Anchor compatibility (совместимость anchor cells):
   - Проверка, хватает ли разницы между anchor cells для требуемых шагов
   - Если нет — предупреждение + рекомендации

3. Step size capping:
   - Если шаг был уменьшен из-за нехватки диапазона — это отмечается

---

### 6. Результат (Output)

Функция возвращает словарь CONSTRAINT_CONFIG с финализированными значениями:

{
    'cell_min': 0.0,
    'cell_max': 0.24,               # Adaptive
    'min_step_rating': 0.012,       # Merit pool based
    'step_max_rating': 0.042,
    'min_step_cr': 0.006,
    'step_max_cr': 0.018,
    'rating_multiplier': {0: (4, 1.4)}  # Soft requirements
}


---

### 7. Интерактивный отчёт (Interactive Reporting)

В консоль выводится:
- Размер merit pool
- Диапазон ячеек (cell range span)
- Минимальные требуемые шаги по рейтингам / CR
- Статус реализуемости (Good / Warning)
- Процент использования диапазона
- Проверка совместимости anchor cells

Design Goal: Полная прозрачность — пользователь не только видит результаты, но и понимает, почему они такими получились.

In [1]:
def auto_configure_constraints(num_ratings: int, num_cr_bins: int, merit_pool_pct: float) -> Dict:
    """
    Auto-generate adaptive constraints that balance differentiation with feasibility.
    
    KEY PRINCIPLE: Constraints should guide toward good solutions, not prevent them.
    
    Design Philosophy:
    - Start with SMALL minimum steps to ensure visibility (not maximal differentiation)
    - Let cell_max be generous (optimizer will find the right values)
    - Step requirements should create clear ordering, not force huge gaps
    - Budget feasibility is paramount - constraints should help, not hinder
    
    Args:
        num_ratings: Number of rating levels (e.g., 5)
        num_cr_bins: Number of CR bins (e.g., 5)
        merit_pool_pct: Average merit pool as decimal (e.g., 0.08 for 8%)
    
    Returns:
        Dictionary of constraint parameters
    """
    config = CONSTRAINT_CONFIG.copy()
    
    # ============================================================================
    # CELL BOUNDS (Global Min/Max for any cell in the matrix)
    # ============================================================================
    
    # Minimum: Default to 0% unless user overrides
    if config['cell_min'] is None:
        config['cell_min'] = 0.0
    
    # Maximum: Should be generous but realistic
    # This is an UPPER BOUND, not a target - optimizer can use less
    if config['cell_max'] is None:
        # Scale based on merit pool size
        # Smaller pools need higher multipliers for any differentiation
        # But keep it reasonable - this is a ceiling, not a requirement
        if merit_pool_pct <= 0.03:      # ≤3% pools (very tight)
            multiplier = 4.0
        elif merit_pool_pct <= 0.05:    # 3-5% pools
            multiplier = 3.5
        elif merit_pool_pct <= 0.08:    # 5-8% pools
            multiplier = 3.0 
        elif merit_pool_pct <= 0.12:    # 8-12% pools
            multiplier = 2.5 # 2.5
        elif merit_pool_pct <= 0.20:    # 12-20% pools
            multiplier = 2 # 2
        else:                           # >20% pools
            multiplier = 1.5 # 1.5
        
        config['cell_max'] = merit_pool_pct * multiplier
    
    # ============================================================================
    # STEP SIZES: Based on MERIT POOL, not theoretical span
    # ============================================================================
    # 
    # Key Insight: If average merit is 8%, we might want something like:
    #   Rating 1: 2%,  Rating 2: 5%,  Rating 3: 8%,  Rating 4: 11%,  Rating 5: 14%
    # 
    # That's about 3% steps (0.03) with a total range of 12% (0.12)
    # This is MUCH smaller than the theoretical span of (0% to 24% = 0.24)
    # 
    # The old approach would force steps based on 0.24, creating impossible constraints.
    # New approach: Base steps on merit_pool_pct for realistic, achievable targets.
    # ============================================================================
    
    num_rating_steps = max(num_ratings - 1, 1)
    num_cr_steps = max(num_cr_bins - 1, 1)
    
    # ============================================================================
    # RATING STEP SIZES (Horizontal: across ratings within same CR bin)
    # ============================================================================
    
    if config['min_step_rating'] is None:
        # Base calculation: Use merit pool as reference, not theoretical span
        # 
        # For visibility, we want at least 10-15% of the merit pool per step
        # Example: 8% pool → 0.8% to 1.2% minimum step
        # This ensures Rating 5 is clearly better than Rating 1, but doesn't over-constrain
        
        # Base step as percentage of merit pool
        base_step_factor = 0.15  # 15% of merit pool per step (modest)
        
        config['min_step_rating'] = merit_pool_pct * base_step_factor
        
        # Adjust for number of ratings - more ratings need smaller steps
        if num_ratings >= 7:
            config['min_step_rating'] *= 0.75  # 25% smaller for 7+ ratings
        
        # Absolute floor: Never less than 0.5% for visibility
        config['min_step_rating'] = max(config['min_step_rating'], 0.005)
        
        # Absolute ceiling: Never more than 3% (prevents excessive differentiation)
        config['min_step_rating'] = min(config['min_step_rating'], 0.03)
    
    if config['step_max_rating'] is None:
        # Maximum step: Allow some flexibility but prevent spiky matrices
        # Roughly 3-4x the minimum step
        config['step_max_rating'] = config['min_step_rating'] * 3.5
        
        # Hard cap: Never more than 8% in a single step
        config['step_max_rating'] = min(config['step_max_rating'], 0.08)
    
    # ============================================================================
    # CR STEP SIZES (Vertical: across CR bins within same rating)
    # ============================================================================
    
    if config['min_step_cr'] is None:
        # CR steps should be somewhat smaller than rating steps
        # Rationale: Performance (rating) is primary differentiator, CR is secondary
        # Typically 40-60% of rating step size
        
        config['min_step_cr'] = config['min_step_rating'] * 0.5
        
        # Adjust for number of bins
        if num_cr_bins >= 7:
            config['min_step_cr'] *= 0.75
        
        # Floor: Never less than 0.3% for visibility
        config['min_step_cr'] = max(config['min_step_cr'], 0.003)
        
        # Ceiling: Never more than 2%
        config['min_step_cr'] = min(config['min_step_cr'], 0.02)
    
    if config['step_max_cr'] is None:
        # Maximum CR step
        config['step_max_cr'] = config['min_step_cr'] * 3.0
        
        # Hard cap: Never more than 5%
        config['step_max_cr'] = min(config['step_max_cr'], 0.05)
    
    # ============================================================================
    # MULTIPLIER CONSTRAINTS (Rating-to-rating ratio requirements)
    # ============================================================================
    
    if config['rating_multiplier'] is None:
        multipliers = {}
        
        # Soften multiplier requirements - just ensure clear ordering
        if num_ratings >= 2:
            # Top rating should be notably higher than bottom, but not extreme
            # Old: 1.8x, New: 1.4x (more achievable)
            lowest_idx = 0
            highest_idx = num_ratings - 1
            multipliers[lowest_idx] = (highest_idx, 1.4)
        
        if num_ratings >= 4:
            # If we have at least 4 ratings, ensure second-from-top is clearly above bottom
            # Old: (1, highest, 1.4), New: (0, second_highest, 1.25)
            multipliers[0] = (num_ratings - 2, 1.25)
        
        config['rating_multiplier'] = multipliers
    
    # ============================================================================
    # VALIDATION & DIAGNOSTICS
    # ============================================================================
    
    # Calculate what the constraints imply about total range
    min_range_rating = config['min_step_rating'] * num_rating_steps
    min_range_cr = config['min_step_cr'] * num_cr_steps
    max_theoretical_range = config['cell_max'] - config['cell_min']
    
    print(f"\nConstraint Analysis:")
    print(f"  Merit Pool: {merit_pool_pct*100:.2f}%")
    print(f"  Cell Range: {config['cell_min']*100:.2f}% to {config['cell_max']*100:.2f}% (span: {max_theoretical_range*100:.2f}%)")
    print(f"  Min Rating Step: {config['min_step_rating']*100:.2f}% → Total: {min_range_rating*100:.2f}%")
    print(f"  Min CR Step: {config['min_step_cr']*100:.2f}% → Total: {min_range_cr*100:.2f}%")
    print(f"  Feasibility Check: ", end="")
    
    # Check if constraints are achievable
    if min_range_rating > max_theoretical_range * 0.9:
        print("⚠ WARNING: Rating step requirements are very tight!")
        print(f"    Required range: {min_range_rating*100:.2f}%, Available: {max_theoretical_range*100:.2f}%")
        print(f"    Consider: reducing min_step_rating or increasing cell_max")
    elif min_range_cr > max_theoretical_range * 0.9:
        print("⚠ WARNING: CR step requirements are very tight!")
        print(f"    Required range: {min_range_cr*100:.2f}%, Available: {max_theoretical_range*100:.2f}%")
    else:
        # Calculate utilization percentage
        utilization = max(min_range_rating, min_range_cr) / max_theoretical_range * 100
        print(f"✓ Good ({utilization:.0f}% utilization of available range)")
    
    # Check anchor cell compatibility
    if ANCHOR_CELL_MIN is not None and ANCHOR_CELL_MAX is not None:
        anchor_span = ANCHOR_CELL_MAX - ANCHOR_CELL_MIN
        required_span = max(min_range_rating, min_range_cr)
        
        if anchor_span < required_span * 0.9:
            print(f"\n⚠ WARNING: Anchor cells define span of {anchor_span*100:.2f}%")
            print(f"    but constraints require {required_span*100:.2f}%")
            print(f"    This WILL cause infeasibility!")
            print(f"    Solutions:")
            print(f"      1. Widen anchor range (min={ANCHOR_CELL_MIN}, max={ANCHOR_CELL_MAX})")
            print(f"      2. Set min_step_rating={required_span/num_rating_steps*0.8:.4f}")
            print(f"      3. Set min_step_cr={required_span/num_cr_steps*0.8:.4f}")
    
    return config

def generate_stress_scenarios(num_ratings: int, target_dist: Dict, honor_zeros: bool = False) -> Dict[str, Dict[int, float]]:
    """Generate stress test scenarios dynamically"""
    scenarios = {'target': target_dist}
    
    ratings = list(range(1, num_ratings + 1))
    
    # Inflated: skew toward high ratings
    inflated = {}
    for i, r in enumerate(ratings):
        weight = (i + 1) ** 2
        inflated[r] = weight
    total = sum(inflated.values())
    inflated = {r: v/total for r, v in inflated.items()}
    
    # Harsh: skew toward low ratings
    harsh = {}
    for i, r in enumerate(ratings):
        weight = (num_ratings - i) ** 2
        harsh[r] = weight
    total = sum(harsh.values())
    harsh = {r: v/total for r, v in harsh.items()}
    
    # Forced curve: bell curve
    forced = {}
    for i, r in enumerate(ratings):
        forced[r] = norm.pdf(i, loc=(num_ratings-1)/2, scale=num_ratings/4)
    total = sum(forced.values())
    forced = {r: v/total for r, v in forced.items()}
    
    # Top heavy: more in upper half
    top_heavy = {}
    for i, r in enumerate(ratings):
        if i >= num_ratings // 2:
            top_heavy[r] = 0.7 / (num_ratings - num_ratings // 2)
        else:
            top_heavy[r] = 0.3 / (num_ratings // 2)
    
    scenarios.update({
        'inflated': inflated,
        'harsh': harsh,
        'forced_curve': forced,
        'top_heavy': top_heavy
    })
    
    # If honoring zeros, mask and renormalize all stress scenarios
    if honor_zeros:
        scenarios = {k: mask_zeros_in_distribution(v, target_dist) for k, v in scenarios.items()}
    
    return scenarios


def validate_scenario_distributions(scenarios_dict: Dict[str, Dict[int, float]]) -> None:
    """Validate that scenario distributions are properly formatted"""
    expect = set(RATING_ORDER)
    for name, dist in scenarios_dict.items():
        if set(dist.keys()) != expect:
            raise ValueError(f"Scenario '{name}' keys {set(dist.keys())} must match RATING_ORDER {expect}")
        if abs(sum(dist.values()) - 1.0) > 1e-9:
            raise ValueError(f"Scenario '{name}' must sum to 1.0, got {sum(dist.values())}")


# Generate configuration - VALIDATE CUSTOM INPUTS FIRST (fail fast)

# Validate custom target distribution BEFORE using it
validate_custom_target_distribution(CUSTOM_TARGET_DISTRIBUTION, NUM_RATINGS)

# Generate or use custom target distribution
if CUSTOM_TARGET_DISTRIBUTION is not None:
    TARGET_RATING_DISTRIBUTION = CUSTOM_TARGET_DISTRIBUTION.copy()
    print(f"Using custom target distribution")
    # Check if any zeros exist and HARD_ZERO_RATINGS is enabled
    if HARD_ZERO_RATINGS and any(v == 0 for v in TARGET_RATING_DISTRIBUTION.values()):
        zero_ratings = [k for k, v in TARGET_RATING_DISTRIBUTION.items() if v == 0]
        print(f"  Hard zeros enabled: ratings {zero_ratings} will never appear in Monte Carlo")
else:
    TARGET_RATING_DISTRIBUTION = generate_target_distribution(NUM_RATINGS)

# Validate custom CR bins BEFORE using them
validate_custom_cr_bins(CUSTOM_CR_BINS)

# Handle CR bin configuration
if CUSTOM_CR_BINS is not None:
    # Use custom bins and update NUM_CR_BINS accordingly
    CR_BIN_EDGES = list(CUSTOM_CR_BINS)
    NUM_CR_BINS = len(CR_BIN_EDGES) - 1
    print(f"Using custom CR bins: {NUM_CR_BINS} bins defined")
else:
    # Auto-generate bins based on NUM_CR_BINS, CR_BIN_FIRST, CR_BIN_LAST
    CR_BIN_EDGES = generate_cr_bin_edges(NUM_CR_BINS, CR_BIN_FIRST, CR_BIN_LAST)

CONSTRAINTS = auto_configure_constraints(NUM_RATINGS, NUM_CR_BINS, MERIT_POOL_VALUE)
STRESS_SCENARIOS = generate_stress_scenarios(NUM_RATINGS, TARGET_RATING_DISTRIBUTION, honor_zeros=HARD_ZERO_RATINGS)
STRICT_ZERO_MASK = process_force_zero_cells(FORCE_ZERO_CELLS, NUM_CR_BINS, NUM_RATINGS)

# Check if step sizes were capped due to tight constraints
STEP_SIZE_CAPPED = False
span = CONSTRAINTS['cell_max'] - CONSTRAINTS['cell_min']
uncapped_min_rating = CONSTRAINTS['cell_max'] * 0.08 / max(NUM_RATINGS - 1, 1)
if NUM_RATINGS > 5:
    uncapped_min_rating *= 0.7
uncapped_min_cr = uncapped_min_rating * 0.45
if NUM_CR_BINS > 5:
    uncapped_min_cr *= 0.8

if CONSTRAINTS['min_step_rating'] < uncapped_min_rating * 0.99:  # Small tolerance for rounding
    STEP_SIZE_CAPPED = True
if CONSTRAINTS['min_step_cr'] < uncapped_min_cr * 0.99:
    STEP_SIZE_CAPPED = True

# Validate scenarios
validate_scenario_distributions(STRESS_SCENARIOS)

# ======================
# VALIDATION
# ======================

def validate_configuration():
    """Validate user configuration"""
    errors = []
    warnings_list = []
    
    if NUM_RATINGS < 2:
        errors.append("NUM_RATINGS must be at least 2")
    
    if abs(sum(TARGET_RATING_DISTRIBUTION.values()) - 1.0) > 1e-8:
        errors.append(f"TARGET_RATING_DISTRIBUTION must sum to 1.0")
    
    # Validate CR bin configuration consistency (only if auto-generating)
    if CUSTOM_CR_BINS is None:
        if CR_BIN_FIRST >= CR_BIN_LAST:
            errors.append(f"CR_BIN_FIRST must be < CR_BIN_LAST")
        if NUM_CR_BINS < 2:
            errors.append("NUM_CR_BINS must be at least 2")
    
    if MERIT_POOL_VALUE <= 0:
        errors.append("MERIT_POOL_VALUE must be positive")
    
    if BUDGET_TOLERANCE_LOWER <= 0 or BUDGET_TOLERANCE_LOWER >= 1:
        errors.append("BUDGET_TOLERANCE_LOWER must be between 0 and 1")
    
    if BUDGET_TOLERANCE_UPPER <= 0 or BUDGET_TOLERANCE_UPPER >= 1:
        errors.append("BUDGET_TOLERANCE_UPPER must be between 0 and 1")
    
    if SCENARIO_CONCENTRATION <= 0:
        errors.append("SCENARIO_CONCENTRATION must be positive")
    
    # Validate anchor cells
    if ANCHOR_CELL_MAX is not None:
        if ANCHOR_CELL_MAX <= 0:
            errors.append("ANCHOR_CELL_MAX must be positive if specified")
        if ANCHOR_CELL_MAX > 1.0:
            warnings_list.append(f"ANCHOR_CELL_MAX of {ANCHOR_CELL_MAX:.2%} is above 100% - this is unusual")
    
    if ANCHOR_CELL_MIN is not None:
        if ANCHOR_CELL_MIN < 0:
            errors.append("ANCHOR_CELL_MIN cannot be negative")
        if ANCHOR_CELL_MAX is not None and ANCHOR_CELL_MIN >= ANCHOR_CELL_MAX:
            errors.append(f"ANCHOR_CELL_MIN ({ANCHOR_CELL_MIN}) must be < ANCHOR_CELL_MAX ({ANCHOR_CELL_MAX})")
    
    if errors:
        raise ValueError("Configuration errors:\n  " + "\n  ".join(errors))
    
    print("Configuration validated")
    
    # Print zero enforcement info
    num_forced_zeros = STRICT_ZERO_MASK.sum()
    if num_forced_zeros > 0:
        print(f"Strict zero enforcement enabled for {num_forced_zeros} cells:")
        for i in range(NUM_CR_BINS):
            for j in range(NUM_RATINGS):
                if STRICT_ZERO_MASK[i, j]:
                    print(f"  - CR bin {i}, Rating {j+1}: FORCED to 0%")
    
    # Print anchor cell info
    anchors = get_anchor_cells()
    if anchors:
        print(f"Anchor cells PINNED to exact values:")
        for anchor_type, (i, j, value) in anchors.items():
            if anchor_type == 'max':
                print(f"  - CR bin {i}, Rating {j+1} (best performers): PINNED at {value*100:.2f}%")
            else:
                print(f"  - CR bin {i}, Rating {j+1} (worst performers): PINNED at {value*100:.2f}%")
    
    if warnings_list:
        print("\nConfiguration warnings:")
        for w in warnings_list:
            print(f"  - {w}")
    
    # Warn about step size capping
    if STEP_SIZE_CAPPED:
        print("\nStep sizes capped: The range between ANCHOR_CELL_MIN/MAX or small cell_max")
        print("    required reducing min_step_rating or min_step_cr to prevent infeasibility.")


# ======================
# UTILITY FUNCTIONS
# ======================

def round_matrix(matrix: np.ndarray, decimals: int = 4) -> np.ndarray:
    """Round matrix values to human-readable percentages"""
    return np.round(matrix, decimals)


def enforce_strict_zeros(matrix: np.ndarray) -> np.ndarray:
    """Enforce strict zero values for specified cells"""
    matrix = matrix.copy()
    matrix[STRICT_ZERO_MASK] = 0.0
    return matrix


def enforce_all_fixed_cells(matrix: np.ndarray) -> np.ndarray:
    """Enforce both strict zeros and anchor cells"""
    matrix = enforce_strict_zeros(matrix)
    matrix = enforce_anchor_cells(matrix)
    return matrix

NameError: name 'Dict' is not defined

## Data Structures

---

### MatrixCandidate

**Назначение:** Полное представление одного решения матрицы.

**Поля:**
- `matrix` — numpy array [I×J] с процентами merit increase
- `fitness` — итоговый композитный скор (взвешенная сумма budget + constraints + diff)
- `budget_score` — 0..1, насколько хорошо решение попадает в допустимый бюджетный диапазон
- `constraint_score` — 0..1, штраф за нарушения ограничений (монотонность, шаги, границы)
- `success_rate` — % сценариев, в которых бюджет попадает в допустимый диапазон
- `mean_deviation` — среднее |actual - target| / target по всем сценариям
- `generation` — поколение GA, в котором кандидат был создан или улучшен

**Использование:**
- Возвращается из GA evolution
- Сортируется по `fitness` для ранжирования
- Используется для экспорта и аналитики

---

### PolicyGuidance

**Назначение:** Управленческие рекомендации для одного уровня рейтинга.

**Поля:**
- `rating` — номер рейтинга (от 1 до NUM_RATINGS)
- `target_pct` — целевой % сотрудников на этом рейтинге (из TARGET_RATING_DISTRIBUTION)
- `hard_min_pct` — P10 успешных распределений (консервативный нижний порог)
- `hard_max_pct` — P90 успешных распределений (консервативный верхний порог)

**Интерпретация:**
- Руководитель должен стремиться к `target_pct`
- Диапазон `[hard_min_pct, hard_max_pct]` задаёт «зону безопасности» с учётом вариативности
- Если фактическое распределение выходит за границы → повышенный риск превышения бюджета

**Пример:**
Rating 3: target=50%, hard_min=45%, hard_max=55%  
→ Руководители должны стремиться к ~50%, допустимый диапазон 45–55%

**Как генерируется:**
- Рассчитывается после оптимизации для лучшей матрицы
- Использует 200 Monte Carlo сценариев
- Учитывает только успешные сценарии (бюджет в допустимом диапазоне)
- P10 / P90 применяются для устойчивости к выбросам и вариативности

In [ ]:
# ======================
# DATA STRUCTURES
# ======================

@dataclass
class MatrixCandidate:
    matrix: np.ndarray
    fitness: float
    budget_score: float
    constraint_score: float
    success_rate: float
    mean_deviation: float
    generation: int


@dataclass
class PolicyGuidance:
    rating: int
    target_pct: float
    hard_min_pct: float
    hard_max_pct: float

## Загрузка данных (Data Loading)

### load_employee_data()

**Назначение:** Загрузить популяцию сотрудников для построения salary matrix.

**Варианты входных данных:**

**1. Excel-файл (если filepath указан):**
- Обязательные столбцы: `base_salary`, `CR`
- Необязательные: `employee_id`, другие метаданные
- Переименование: `CR` → `compa_ratio` для единообразия

**2. Синтетические данные (если filepath=None или файл не найден):**
- Генерируется N=1200 сотрудников
- `base_salary` — логнормальное распределение (mean=11.1, sigma=0.35)  
  → реалистичное распределение зарплат с «правым хвостом»
- `compa_ratio` — normal(1.0, 0.10), обрезка до [0.6, 1.4]  
  → центрировано на значение рынка (CR=1.0)

**Обработка:**
1. Загрузка или генерация данных
2. Создание столбца `cr_bin` через `pd.cut`:
   - Используются `CR_BIN_EDGES`
   - `include_lowest=True` — включает минимальное значение в первый bin
   - `right=False` — интервалы в формате [low, high)
3. Нумерация bin’ов: 0, 1, ..., NUM_CR_BINS - 1

**Вывод:**
- DataFrame со столбцами:  
  `employee_id`, `base_salary`, `compa_ratio`, `cr_bin`
- Готов для генерации сценариев

**Design Note:**
- Синтетические данные используются для тестов и демо
- В продакшене ожидаются реальные данные
- Биннинг по CR должен соответствовать структуре матрицы

---

In [ ]:
# ======================
# DATA LOADING
# ======================

def load_employee_data(filepath: Optional[str] = None) -> pd.DataFrame:
    """Load employee data from Excel or generate synthetic"""
    if filepath is not None:
        if not os.path.exists(filepath):
            print(f"Warning: File '{filepath}' not found. Using synthetic data.")
            filepath = None
        else:
            try:
                df = pd.read_excel(filepath)
                if 'base_salary' not in df.columns or 'CR' not in df.columns:
                    raise ValueError("Excel must contain 'base_salary' and 'CR' columns")
                df = df.rename(columns={'CR': 'compa_ratio'})
                print(f"Loaded data from '{filepath}'")
            except Exception as e:
                print(f"Error loading '{filepath}': {e}. Using synthetic data.")
                filepath = None
    
    if filepath is None:
        np.random.seed(SEED_BASE_POPULATION)
        n = 1200
        df = pd.DataFrame({
            'employee_id': np.arange(n) + 1,
            'base_salary': np.random.lognormal(mean=11.1, sigma=0.35, size=n),
            'compa_ratio': np.clip(np.random.normal(loc=1.0, scale=0.10, size=n), 0.6, 1.4),
        })
        print(f"Generated synthetic data")
    
    df['cr_bin'] = pd.cut(df['compa_ratio'], bins=CR_BIN_EDGES,
                          labels=np.arange(NUM_CR_BINS),
                          include_lowest=True, right=False).astype(int)
    
    return df

## Генерация сценариев (Monte Carlo Simulation)

### generate_rating_scenarios()

**Назначение:**  
Создать множество возможных сценариев распределения рейтингов сотрудников и построить по ним salary-матрицы, чтобы протестировать устойчивость merit matrix.

**Входные параметры:**
- `df` — данные сотрудников  
- `num_scenarios` — сколько сценариев сгенерировать  
- `include_stress` — добавлять ли заранее подготовленные стресс-сценарии

**Процесс:**
1. Если `include_stress=True`, добавляются экстремальные сценарии (`inflated`, `harsh`, `forced_curve`, `top_heavy`), чтобы покрыть крайние случаи.
2. Остальные сценарии создаются через Dirichlet-семплирование вокруг `TARGET_RATING_DISTRIBUTION`.
   - Параметр `SCENARIO_CONCENTRATION` управляет вариативностью:
     - Выше → ближе к target (меньше разброс)
     - Ниже → больше разброс (стресс-тест)
3. Если `HARD_ZERO_RATINGS=True`, рейтинги с нулевой вероятностью полностью исключаются из семплирования.  
   Если False, им дается очень малая (epsilon) вероятность.

**Результат:**  
Список salary-матриц (по одной на сценарий), готовых для векторной оценки fitness.

---

### create_salary_matrix()

**Назначение:**  
Преобразовать одно распределение рейтингов в salary matrix размером `[CR bins × Ratings]`.

**Используемый (оптимизированный) подход:**
- Каждому сотруднику присваивается рейтинг вероятностно (векторно).
- Рейтинг преобразуется в индекс столбца матрицы.
- Выполняется groupby по `cr_bin` и `rating_idx` с суммированием `base_salary`.
- Результат заполняет соответствующие ячейки матрицы.

**Преимущества:**
- Один проход по данным (O(N))
- Без вложенных циклов
- Векторные операции + groupby
- ~100–500× быстрее на больших данных

**Интерпретация результата:**
- `mat[i, j]` = общая сумма base_salary сотрудников с CR bin `i` и rating `j`
- `0` в ячейке означает отсутствие сотрудников в этой комбинации

---

### Zero Support Mask

**compute_zero_support_mask():**
- Находит ячейки, где зарплата равна 0 во всех сценариях.
- Такие ячейки можно зафиксировать в 0%, так как они не влияют на бюджет.

**combined_zero_mask = zero_support_mask OR STRICT_ZERO_MASK**
- Объединяет:
  - нули, основанные на данных (нет сотрудников),
  - принудительные нули от пользователя.
- Применяется ко всем матрицам для консистентности.

In [ ]:
# ======================
# SCENARIO GENERATION
# ======================

def generate_rating_scenarios(df: pd.DataFrame, num_scenarios: int, 
                             include_stress: bool = True) -> List[np.ndarray]:
    """Generate Monte Carlo rating scenarios"""
    rng = np.random.default_rng(SEED_SCENARIOS)
    scenarios = []
    
    if include_stress:
        for dist in STRESS_SCENARIOS.values():
            S = create_salary_matrix(df, dist, rng)
            scenarios.append(S)
    
    remaining = max(0, num_scenarios - len(scenarios))
    base_probs = np.array([TARGET_RATING_DISTRIBUTION[r] for r in sorted(RATING_ORDER)])
    
    for _ in range(remaining):
        perturbed = dirichlet_sample_from_target(base_probs, SCENARIO_CONCENTRATION, rng)
        perturbed_dist = {r: p for r, p in zip(sorted(RATING_ORDER), perturbed)}
        S = create_salary_matrix(df, perturbed_dist, rng)
        scenarios.append(S)
    
    return scenarios


def create_salary_matrix(df: pd.DataFrame, rating_dist: Dict, rng) -> np.ndarray:
    """Create salary matrix for a specific rating distribution (optimized with groupby)"""
    df_temp = df.copy()
    df_temp['rating'] = rng.choice(
        RATING_ORDER,
        size=len(df_temp),
        p=[rating_dist[r] for r in sorted(RATING_ORDER)]
    )
    rating_to_idx = {r: i for i, r in enumerate(sorted(RATING_ORDER))}
    df_temp['rating_idx'] = df_temp['rating'].map(rating_to_idx)
    
    # Fast aggregate using groupby (O(N) instead of O(N*I*J))
    agg = (df_temp
           .groupby(['cr_bin', 'rating_idx'], as_index=False)['base_salary']
           .sum())
    
    mat = np.zeros((NUM_CR_BINS, NUM_RATINGS))
    mat[agg['cr_bin'].values, agg['rating_idx'].values] = agg['base_salary'].values
    
    return mat


## Оценка пригодности (Fitness Evaluation) — векторизованное скорингование

### evaluate_fitness()

**Назначение:**  
Оценить качество (fitness) матрицы повышений, протестировав её на множестве сценариев зарплат в векторизованном виде (быстро и без циклов).

**Ключевая оптимизация:**  
Все сценарии заранее объединены в один 3D-массив. Это позволяет вычислять итоговую стоимость для всех сценариев одной векторной операцией (без K отдельных циклов), что радикально ускоряет работу.

**Вход:**
- Матрица повышений
- Набор salary-матриц по сценариям
- Целевой бюджет (merit_pool)
- Нижняя и верхняя границы допустимого отклонения бюджета (асимметричные)

**Выход:**
- total_fitness — итоговый скоринг
- budget_score — точность попадания в бюджет
- constraint_score — соблюдение ограничений
- success_rate — % сценариев внутри допустимого бюджета
- mean_deviation — среднее отклонение от бюджета

---

## Компоненты Fitness

### 1. Budget Score (учет асимметричного бюджета)

Для каждого сценария вычисляется отклонение от целевого бюджета в процентах и проверяется, лежит ли оно в допустимом диапазоне.  
Затем применяется асимметричная штрафная функция:
- перерасход наказывается сильнее, чем недорасход (если так задано),
- внутри диапазона штраф = 0,
- вне диапазона штраф растет пропорционально величине нарушения.

Далее штрафы усредняются, и итоговый `budget_score` рассчитывается по формуле:

**1 / (1 + 20 × средний штраф)**  
→ от 1 (идеально) до 0 (очень плохо)

### 2. Constraint Score (структурные ограничения)

Проверяются все заложенные ограничения. Каждое нарушение добавляет штраф с разным весом.

Типы ограничений:
- **Строгие нули** — ячейки, которые должны быть = 0. Любое отклонение штрафуется максимально.
- **Anchor cells** — фиксированные значения (например, лучшие и худшие ячейки). Любое отклонение = огромный штраф.
- **Монотонность по рейтингам (горизонталь)** — значения должны расти слева направо.
- **Монотонность по CR (вертикаль)** — значения должны уменьшаться сверху вниз.
- **Границы ячеек** — не ниже минимума и не выше максимума.
- **Шаги по рейтингам и CR** — между соседними ячейками шаг должен быть не слишком маленьким и не слишком большим.

После суммирования штрафов итоговый `constraint_score` =  
**1 / (1 + total_penalty)**

### 3. Differentiation Score (видимая дифференциация)

Матрица должна создавать **заметную разницу** между низкой и высокой эффективностью.

Логика:
- Для каждой строки считаются средние положительные шаги между рейтингами.
- Хорошо, если шаги **чуть выше минимума**, но **не слишком большие** (избегаем резких скачков).
- Нижние CR (основная масса сотрудников) получают больший вес.

Результат: значение от 0 до 1 (чем выше — тем лучше дифференциация).

### 4. Итоговый Fitness

Финальный скоринг — это **взвешенная сумма всех трёх компонентов**:

- Бюджет (основной вес, обычно ~70%)
- Ограничения (важность структуры, ~30%)
- Дифференциация (дополнительный бонус, если включен DIFF_WEIGHT)

Такой подход обеспечивает баланс между:
- точностью соблюдения бюджета,
- корректной структурой матрицы,
- видимой разницей между уровнями производительности.

**Итог:** fitness ∈ [0, 1]  
Чем ближе к 1 — тем лучше решение.

In [ ]:
# ======================
# FITNESS EVALUATION
# ======================

def compute_zero_support_mask(scenarios: List[np.ndarray]) -> np.ndarray:
    """Identify cells with zero payroll across all scenarios"""
    S_stack = np.stack(scenarios)
    return (np.abs(S_stack) <= 1e-12).all(axis=0)


def evaluate_fitness(matrix: np.ndarray, S_stack: np.ndarray, 
                    merit_pool: float, budget_tol_lower: float, budget_tol_upper: float) -> Tuple[float, float, float, float, float]:
    """
    Vectorized fitness evaluation across pre-stacked scenarios
    
    Args:
        matrix: Merit increase matrix [I, J]
        S_stack: Pre-stacked salary matrices [K, I, J]
        merit_pool: Target budget
        budget_tol_lower: Lower tolerance (underspend allowed)
        budget_tol_upper: Upper tolerance (overspend allowed)
    
    Returns: (total_fitness, budget_score, constraint_score, success_rate, mean_deviation)
    """
    # Vectorized cost computation
    costs = (S_stack * matrix).sum(axis=(1, 2))
    deviations = (costs - merit_pool) / merit_pool  # Keep sign for asymmetric check
    
    # Asymmetric budget check: -lower_tol <= deviation <= +upper_tol
    within_budget = ((deviations >= -budget_tol_lower) & (deviations <= budget_tol_upper)).sum()
    success_rate = within_budget / len(S_stack)
    mean_deviation = np.abs(deviations).mean()  # For reporting only
    
    # Asymmetric band-aware budget score (zero penalty inside band, scaled outside)
    def _band_penalty(dev, low, up):
        """Calculate penalty for deviation outside allowed band"""
        if dev < -low:   # underspend beyond allowed
            return (-dev - low) / max(low, 1e-9)
        if dev > up:     # overspend beyond allowed
            return (dev - up) / max(up, 1e-9)
        return 0.0       # inside band → no penalty
    
    penalties = np.array([_band_penalty(d, budget_tol_lower, budget_tol_upper) for d in deviations])
    mean_penalty = penalties.mean()
    budget_score = 1.0 / (1.0 + 20.0 * mean_penalty)
    
    constraint_score = evaluate_constraints(matrix)
    
    # Reward visible rating differentiation (robust slope) alongside budget & constraints
    diff_sc = differentiation_score(matrix)

    total_fitness = (
        FITNESS_WEIGHTS['budget'] * budget_score +
        FITNESS_WEIGHTS['constraints'] * constraint_score +
        DIFF_WEIGHT * diff_sc
    )

    return total_fitness, budget_score, constraint_score, success_rate, mean_deviation


def evaluate_constraints(matrix: np.ndarray) -> float:
    """Evaluate constraint satisfaction with strict zero and anchor cell enforcement"""
    penalties = []
    I, J = matrix.shape
    
    # Small epsilon for floating point comparison
    EPS = 1e-9
    
    # Get anchor cells and precompute positions
    anchors = get_anchor_cells()
    anchor_positions = {(i, j) for _, (i, j, _) in anchors.items()}
    
    # CRITICAL: Heavily penalize violations of strict zero cells
    for i in range(I):
        for j in range(J):
            if STRICT_ZERO_MASK[i, j] and abs(matrix[i, j]) > EPS:
                penalties.append(abs(matrix[i, j]) * 1000)  # Massive penalty
    
    # CRITICAL: Heavily penalize deviations from anchor cells
    for anchor_type, (i, j, target_value) in anchors.items():
        if abs(matrix[i, j] - target_value) > EPS:
            penalties.append(abs(matrix[i, j] - target_value) * 1000)  # Massive penalty
    
    # Monotonicity violations (rating increases)
    for i in range(I):
        for j in range(J - 1):
            # Skip monotonicity check across forced zeros
            if STRICT_ZERO_MASK[i, j] or STRICT_ZERO_MASK[i, j+1]:
                continue
            if matrix[i, j+1] < matrix[i, j]:
                penalties.append((matrix[i, j] - matrix[i, j+1]) * 10)
    
    # Monotonicity violations (CR decreases)
    for i in range(I - 1):
        for j in range(J):
            # Skip monotonicity check across forced zeros
            if STRICT_ZERO_MASK[i, j] or STRICT_ZERO_MASK[i+1, j]:
                continue
            if matrix[i+1, j] > matrix[i, j]:
                penalties.append((matrix[i+1, j] - matrix[i, j]) * 10)
    
    # Cell bounds
    for i in range(I):
        for j in range(J):
            if STRICT_ZERO_MASK[i, j]:
                continue  # Skip bound checks for forced zeros
            # Skip bound checks for anchor cells (they have exact values)
            if (i, j) in anchor_positions:
                continue
            if matrix[i, j] < CONSTRAINTS['cell_min']:
                penalties.append((CONSTRAINTS['cell_min'] - matrix[i, j]) * 20)
            if matrix[i, j] > CONSTRAINTS['cell_max']:
                penalties.append((matrix[i, j] - CONSTRAINTS['cell_max']) * 20)
    
    # Rating step violations
    for i in range(I):
        for j in range(J - 1):
            # Skip step check across forced zeros
            if STRICT_ZERO_MASK[i, j] or STRICT_ZERO_MASK[i, j+1]:
                continue
            step = matrix[i, j+1] - matrix[i, j]
            if step < CONSTRAINTS['min_step_rating']:
                penalties.append((CONSTRAINTS['min_step_rating'] - step) * 5)
            if step > CONSTRAINTS['step_max_rating']:
                penalties.append((step - CONSTRAINTS['step_max_rating']) * 5)
    
    # CR step violations
    for i in range(I - 1):
        for j in range(J):
            # Skip step check across forced zeros
            if STRICT_ZERO_MASK[i, j] or STRICT_ZERO_MASK[i+1, j]:
                continue
            step = matrix[i, j] - matrix[i+1, j]
            if step < CONSTRAINTS['min_step_cr']:
                penalties.append((CONSTRAINTS['min_step_cr'] - step) * 5)
            if step > CONSTRAINTS['step_max_cr']:
                penalties.append((step - CONSTRAINTS['step_max_cr']) * 5)
    
    total_penalty = sum(penalties)
    score = 1.0 / (1.0 + total_penalty)
    
    return score


def differentiation_score(matrix: np.ndarray) -> float:
    """
    Reward steeper (but reasonable) increases across ratings per CR row.
    Returns 0..1 where higher means more visible, well-behaved differentiation.
    Design:
      - Uses average positive step across ratings (skip forced zeros).
      - Targets slightly above min_step_rating; caps to avoid spikes.
      - Weights lower-CR rows a bit more (visibility to core population).
    """
    I, J = matrix.shape
    per_row_scores = []

    for i in range(I):
        steps = []
        for j in range(J - 1):
            # skip around forced zeros (they break smoothness by design)
            if STRICT_ZERO_MASK[i, j] or STRICT_ZERO_MASK[i, j + 1]:
                continue
            step = matrix[i, j + 1] - matrix[i, j]
            steps.append(max(step, 0.0))  # only reward upward steps

        if not steps:
            per_row_scores.append(0.0)
            continue

        mean_step = float(np.mean(steps))

        # Gentle target and cap:
        target = CONSTRAINTS['min_step_rating'] * 1.15  # ask a bit more than minimum
        cap    = CONSTRAINTS['step_max_rating'] * 0.5   # avoid spiky rows

        # Normalize: 0 at 0 step; ~1 when at/above target (clipped by cap)
        norm = min(mean_step, cap) / max(target, 1e-9)
        per_row_scores.append(max(0.0, min(1.0, norm)))

    # Slightly favor low-CR rows (more visible to business)
    # Row 0 weight=1.0 → last row ~0.6
    if I > 1:
        weights = np.linspace(1.0, 0.6, I)
        score = float(np.average(per_row_scores, weights=weights))
    else:
        score = float(np.mean(per_row_scores))

    return score

## Генетические операторы — механика эволюции

### 1. Инициализация популяции

**Стратегия:** комбинировать разные типы матриц, чтобы сразу получить разнообразие и избегать тупиков.

- ~70% — структурированные матрицы (гладкие, соблюдают ограничения)
- ~30% — структурированные, но с агрессивными нулями (исследуют политики «zero merit»)

**Почему так:**
- Полностью случайные матрицы → почти всегда нарушают ограничения и только тратят время
- Только структурированные → высокий риск застрять в локальном оптимуме
- Смешанный подход → баланс между исследованием (exploration) и улучшением (exploitation)

---

### 2. Создание структурированных матриц

**Логика построения:**
1. Начать с лучшей ячейки (низкий CR, высокий рейтинг).
   - Если задан anchor — использовать его.
   - Иначе взять случайное значение в верхнем диапазоне.
2. Заполнить верхнюю строку слева направо, постепенно уменьшая значения.
3. Заполнить остальные строки сверху вниз, также уменьшая значения.
4. Привести значения к допустимым границам, округлить и применить фиксированные ячейки.

**Почему это работает:**
- Начинаем с максимального значения для лучших сотрудников.
- Плавное и монотонное снижение даёт реалистичную форму.
- Случайные шаги обеспечивают разнообразие.

**Вариант с нулями:**
- Использует ту же логику,
- но дополнительно «приглушает» низкие рейтинги в высоких CR-бинах (например, 40% шанс обнулить).
- Это исследует сценарии, где слабые исполнители получают 0%.

---

### 3. Селекция (выбор родителей)

Используется **tournament selection**:
- Берём k случайных кандидатов,
- Выбираем лучшего из них.

**Плюсы:**
- Простая и быстрая стратегия,
- Управляемое давление (чем больше k — тем агрессивнее отбор),
- Сохраняет разнообразие, если k небольшое.

---

### 4. Кроссовер (смешивание родителей)

**Принцип:** смешивание (blend), а не жесткое переключение.

- Потомок = взвешенная комбинация двух родителей.
- Это сохраняет форму обеих матриц и создаёт плавный результат.

**Дополнительная обработка потомка:**
- Сглаживание (убрать шум),
- Округление (для читаемости),
- Применение фиксированных ячеек (нельзя ломать нули или anchor’ы).

**Почему смешивание лучше:**
- Матрица — непрерывное пространство,
- blend даёт реалистичную структуру,
- дискретное наследование дало бы «ломаные» и хаотичные матрицы.

---

### 5. Мутация

Вносит небольшие случайные изменения в ячейки.

**Как работает:**
- С вероятностью `mutation_rate` ячейка слегка изменяется (маленький Gaussian шум).
- Никогда не изменяются:
  - принудительные нули,
  - anchor-ячейки.

**После мутации:** всегда выполняется восстановление структуры (repair), чтобы не нарушать ограничения.

---

### 6. Repair (восстановление структуры)

Исправляет нарушения после мутации или кроссовера.

**Шаги восстановления:**
1. Ограничить значения по минимальному/максимальному порогу.
2. Обеспечить рост по рейтингам (слева направо).
3. Обеспечить спад по CR (сверху вниз).
4. Снова ограничить значения (если вышли за границы).
5. Округлить и вернуть фиксированные ячейки в исходное состояние.

**Важно:**  
Проверки не выполняются через ячейки с принудительными нулями, так как они создают «разрывы» по дизайну.

---

### 7. Smoothing (сглаживание)

- Убирает шум и резкие скачки,
- Делает матрицу визуально и бизнес-логически приятнее,
- Небольшое сглаживание (σ ≈ 0.5) сохраняет форму, но делает её более плавной.

**Баланс:**
- Слишком сильное сглаживание → потеря дифференциации,
- Слишком слабое → много шума,
- Подобран оптимальный уровень.

---

## Локальная донастройка (Local Refinement)

### local_refinement()

**Назначение:**  
Дополировать лучшие матрицы после работы генетического алгоритма с помощью градиентного оптимизатора.

**Когда применяется:**
- Только к топ-5 кандидатов (чтобы не тратить время на слабые решения),
- После завершения GA,
- Используется небольшой набор сценариев (например, 100) для ускорения.

**Метод:** L-BFGS-B  
Почему:
- Поддерживает границы для каждой ячейки,
- Эффективен для больших задач,
- Быстро сходится,
- Не требует явного градиента (вычисляется численно).

**Что происходит с матрицей внутри оптимизации:**
- При каждом шаге матрица ремонтируется (repair),
- Применяются фиксированные ячейки (anchor и strict zeros),
- Вычисляется fitness,
- Оптимизатор пытается максимизировать fitness (минимизируя его отрицание).

**Ограничения:**
- Принудительные нули → ячейка закреплена в 0,
- Anchor ячейки → фиксированы на заданных значениях,
- Остальные — в пределах `cell_min`/`cell_max`.

**Безопасность:**
- Максимум 100 итераций,
- Если улучшения нет или оптимизация не удалась — оставляется исходная матрица.

**Плюсы:**
- Может улучшить fitness ещё на 0.5–2%,
- Убирает мелкие артефакты после GA,
- Извлекает максимум из найденной структуры.

**Минусы:**
- Дорого по времени,
- Может переобучиться, если применять к слабым решениям.

**Компромисс (идеальный):**
- Применять только к лучшим кандидатам,
- Использовать ограниченное число сценариев,
- Сохранять исходную матрицу, если оптимизация не улучшила результат.


In [ ]:
# ======================
# GENETIC OPERATORS
# ======================

def initialize_population(pop_size: int, rng) -> List[np.ndarray]:
    """Initialize population with structured matrices"""
    population = []
    num_with_zeros = int(pop_size * 0.3) if CONSTRAINTS['cell_min'] == 0 else 0

    for i in range(pop_size):
        if i < num_with_zeros:
            matrix = create_structured_matrix_with_zeros(rng)
        else:
            matrix = create_structured_matrix(rng)
        matrix = enforce_all_fixed_cells(matrix)  # Apply all fixed cell constraints
        population.append(matrix)
    return population


def create_structured_matrix(rng) -> np.ndarray:
    """Create a matrix that approximately respects structure"""
    I, J = NUM_CR_BINS, NUM_RATINGS
    matrix = np.zeros((I, J))
    
    # Start from best performer cell (lowest CR, highest rating)
    anchors = get_anchor_cells()
    if 'max' in anchors:
        _, _, value = anchors['max']
        matrix[0, J-1] = value  # Use exact anchor value
    else:
        matrix[0, J-1] = rng.uniform(CONSTRAINTS['cell_max'] * 0.6, CONSTRAINTS['cell_max'])
    
    # Fill top row (lowest CR bin) from right to left
    for j in range(J-2, -1, -1):
        step = rng.uniform(CONSTRAINTS['min_step_rating'], CONSTRAINTS['step_max_rating'])
        matrix[0, j] = max(CONSTRAINTS['cell_min'], matrix[0, j+1] - step)
    
    # Fill down columns (higher CR bins)
    for i in range(1, I):
        for j in range(J):
            step = rng.uniform(CONSTRAINTS['min_step_cr'], CONSTRAINTS['step_max_cr'])
            matrix[i, j] = max(CONSTRAINTS['cell_min'], matrix[i-1, j] - step)
    
    matrix = np.clip(matrix, CONSTRAINTS['cell_min'], CONSTRAINTS['cell_max'])
    matrix = round_matrix(matrix)
    matrix = enforce_all_fixed_cells(matrix)
    
    return matrix


def create_structured_matrix_with_zeros(rng) -> np.ndarray:
    """Create matrix with strategic zeros"""
    I, J = NUM_CR_BINS, NUM_RATINGS
    matrix = np.zeros((I, J))
    
    # Start from best performer cell (lowest CR, highest rating)
    anchors = get_anchor_cells()
    if 'max' in anchors:
        _, _, value = anchors['max']
        matrix[0, J-1] = value  # Use exact anchor value
    else:
        matrix[0, J-1] = rng.uniform(CONSTRAINTS['cell_max'] * 0.6, CONSTRAINTS['cell_max'])
    
    # Fill top row (lowest CR bin) from right to left
    for j in range(J-2, -1, -1):
        step = rng.uniform(CONSTRAINTS['min_step_rating'], CONSTRAINTS['step_max_rating'])
        matrix[0, j] = max(CONSTRAINTS['cell_min'], matrix[0, j+1] - step)
    
    # Fill down each column (higher CR bins)
    for i in range(1, I):
        for j in range(J):
            step = rng.uniform(CONSTRAINTS['min_step_cr'], CONSTRAINTS['step_max_cr'])
            matrix[i, j] = max(CONSTRAINTS['cell_min'], matrix[i-1, j] - step)
            
            # Aggressively push lowest performers at highest CR toward zero
            if i >= I - 2 and j <= 1:
                if rng.random() < 0.4:
                    matrix[i, j] = 0.0
    
    matrix = np.clip(matrix, CONSTRAINTS['cell_min'], CONSTRAINTS['cell_max'])
    matrix = round_matrix(matrix)
    matrix = enforce_all_fixed_cells(matrix)
    
    return matrix


def tournament_selection(population: List[MatrixCandidate], k: int, rng) -> MatrixCandidate:
    """Select best from k random candidates"""
    k = min(k, len(population))
    indices = rng.choice(len(population), k, replace=True)
    tournament = [population[i] for i in indices]
    return max(tournament, key=lambda x: x.fitness)


def crossover(parent1: np.ndarray, parent2: np.ndarray, rng) -> np.ndarray:
    """Smart crossover that blends matrices"""
    alpha = rng.uniform(0.3, 0.7)
    child = alpha * parent1 + (1 - alpha) * parent2
    child = smooth_matrix(child)
    child = round_matrix(child)
    child = enforce_all_fixed_cells(child)
    return child


def mutate(matrix: np.ndarray, mutation_rate: float, rng) -> np.ndarray:
    """Structure-aware mutation that preserves fixed cells"""
    I, J = matrix.shape
    mutated = matrix.copy()
    
    # Get anchor cell positions
    anchors = get_anchor_cells()
    anchor_positions = {(i, j) for _, (i, j, _) in anchors.items()}
    
    for i in range(I):
        for j in range(J):
            # Never mutate forced zeros or anchor cells
            if STRICT_ZERO_MASK[i, j] or (i, j) in anchor_positions:
                continue
            if rng.random() < mutation_rate:
                delta = rng.normal(0, CONSTRAINTS['cell_max'] * 0.05)
                mutated[i, j] += delta
    
    mutated = repair_matrix(mutated)
    mutated = round_matrix(mutated)
    mutated = enforce_all_fixed_cells(mutated)
    
    return mutated


def smooth_matrix(matrix: np.ndarray) -> np.ndarray:
    """Apply smoothing to reduce noise"""
    from scipy.ndimage import gaussian_filter
    smoothed = gaussian_filter(matrix, sigma=0.5, mode='nearest')
    smoothed = np.clip(smoothed, CONSTRAINTS['cell_min'], CONSTRAINTS['cell_max'])
    smoothed = enforce_all_fixed_cells(smoothed)
    return smoothed


def repair_matrix(matrix: np.ndarray) -> np.ndarray:
    """Repair matrix to satisfy monotonicity"""
    I, J = matrix.shape
    repaired = matrix.copy()
    
    repaired = np.clip(repaired, CONSTRAINTS['cell_min'], CONSTRAINTS['cell_max'])
    
    # Enforce rating monotonicity
    for i in range(I):
        for j in range(1, J):
            if STRICT_ZERO_MASK[i, j] or STRICT_ZERO_MASK[i, j-1]:
                continue  # Don't repair around forced zeros
            if repaired[i, j] < repaired[i, j-1]:
                if repaired[i, j-1] > CONSTRAINTS['cell_min']:
                    repaired[i, j] = repaired[i, j-1] + CONSTRAINTS['min_step_rating'] * 0.5
                else:
                    repaired[i, j] = CONSTRAINTS['cell_min']
    
    # Enforce CR monotonicity
    for i in range(1, I):
        for j in range(J):
            if STRICT_ZERO_MASK[i, j] or STRICT_ZERO_MASK[i-1, j]:
                continue  # Don't repair around forced zeros
            if repaired[i, j] > repaired[i-1, j]:
                new_val = repaired[i-1, j] - CONSTRAINTS['min_step_cr'] * 0.5
                repaired[i, j] = max(CONSTRAINTS['cell_min'], new_val)
    
    repaired = np.clip(repaired, CONSTRAINTS['cell_min'], CONSTRAINTS['cell_max'])
    repaired = round_matrix(repaired)
    repaired = enforce_all_fixed_cells(repaired)
    
    return repaired


# ======================
# LOCAL REFINEMENT
# ======================

def local_refinement(matrix: np.ndarray, S_stack: np.ndarray, 
                    merit_pool: float, budget_tol_lower: float, budget_tol_upper: float) -> np.ndarray:
    """Apply local optimization with fixed cell constraints"""
    
    def objective(x):
        mat = repair_matrix(x.reshape(NUM_CR_BINS, NUM_RATINGS))
        mat = enforce_all_fixed_cells(mat)
        fitness, _, _, _, _ = evaluate_fitness(mat, S_stack, merit_pool, budget_tol_lower, budget_tol_upper)
        return -fitness
    
    x0 = matrix.flatten()
    bounds = [(CONSTRAINTS['cell_min'], CONSTRAINTS['cell_max'])] * len(x0)
    
    # Override bounds for forced zero cells
    for i in range(NUM_CR_BINS):
        for j in range(NUM_RATINGS):
            idx = i * NUM_RATINGS + j
            if STRICT_ZERO_MASK[i, j]:
                bounds[idx] = (0.0, 0.0)  # Force to stay at 0
    
    # Override bounds for anchor cells
    anchors = get_anchor_cells()
    for _, (i, j, value) in anchors.items():
        idx = i * NUM_RATINGS + j
        bounds[idx] = (value, value)  # Force to stay at exact value
    
    result = minimize(objective, x0, method='L-BFGS-B', bounds=bounds,
                     options={'maxiter': 100, 'disp': False})
    
    if result.success:
        refined = result.x.reshape(NUM_CR_BINS, NUM_RATINGS)
        refined = repair_matrix(refined)
        refined = enforce_all_fixed_cells(refined)
        return refined
    
    return enforce_all_fixed_cells(matrix)

## Основной цикл генетического алгоритма (Main Genetic Algorithm Loop)

### run_genetic_algorithm()

**Главная функция**, которая координирует весь эволюционный процесс: от подготовки данных до формирования финального набора лучших матриц.

---

## Фаза 1: Подготовка и инициализация

### 1.1. Генерация сценариев (Quick Eval)
Создаются сценарии распределения рейтингов (обычно ~2000), включая стресс-сценарии.  
Они используются для быстрой оценки во время эволюции, чтобы не тратить время на полный набор (10k+).

### 1.2. Вычисление Zero Mask
Определяются ячейки, в которых **во всех сценариях** зарплата равна 0.  
Эти ячейки объединяются с **принудительными нулями** (STRICT_ZERO_MASK), формируя `combined_zero_mask`.

### 1.3. Анализ фиксированных ячеек
Система сообщает пользователю:
- Сколько ячеек принудительно зафиксировано (нули + anchor).
- Есть ли конфликт: anchor ячейка, но в данных там нет сотрудников.
Это добавляет прозрачность и предупреждает о потенциальных проблемах.

### 1.4. Инициализация популяции
Создаётся начальный набор матриц по стратегии «структурированные + с нулями».  
К каждой матрице сразу применяется `combined_zero_mask` и все фиксированные ячейки.

### 1.5. Первая оценка fitness
Каждая матрица превращается в объект `MatrixCandidate`  
(содержит матрицу + все метрики fitness).  
Расчитываются:
- fitness,
- budget_score,
- constraint_score,
- success_rate,
- mean_deviation.

---

## Фаза 2: Эволюционный цикл

Алгоритм повторяет поколения, пока не достигнет лимита или не сойдётся.

В каждом поколении выполняется:

### 1. Сортировка по fitness (лучшие — сверху)
Позволяет отслеживать прогресс и выбирать элиту.

### 2. Проверка сходимости (convergence)
Если улучшение fitness меньше порога в течение нескольких поколений подряд → досрочно остановиться (экономия времени).

### 3. Вывод прогресса
Каждые N поколений выводится лучший fitness, success rate и отклонения бюджета.

### 4. Формирование следующего поколения
Создаётся пустой список будущей популяции.

### 5. Элитизм
Лучшие `elite_size` решений копируются **без изменений**.  
Это гарантирует, что хорошие решения не потеряются.

### 6. Создание потомков (offspring)
Пока популяция нового поколения не заполнена:
- **Выбор родителей:** tournament selection.
- **Кроссовер или копирование:** по вероятности `crossover_rate`.
- **Мутация:** случайное изменение отдельных ячеек.
- **Санитизация:** исправление NaN/inf, восстановление структуры (repair).
- **Применение фиксированных ячеек:** zeros и anchors.
- **Оценка fitness:** рассчитываются все компоненты.
- **Добавление в популяцию:** offspring сохраняется как MatrixCandidate.

### 7. Замена популяции
Текущее поколение полностью заменяется новым.

---

## Ключевые механизмы

### Элитизм
- Сохраняет лучшие решения.
- Ускоряет эволюцию.
- Гарантирует, что прогресс не будет потерян.

### Обнаружение сходимости
- Если улучшений нет — ранняя остановка.
- Экономия вычислений.
- Обычно сходимость за 50–200 поколений.

### Принудительное применение фиксированных ячеек
- После **каждой** операции (crossover, mutation, repair).
- Нельзя нарушить anchor или strict zeros.
- Обеспечивает корректность структуры.

### Санитизация
- Если в матрице появились NaN или бесконечности → заменяем валидными значениями.
- Затем выполняем repair.
- Это делает алгоритм устойчивым к числовым артефактам.

---

## Фаза 3: Финальный вывод

После завершения цикла:

- Популяция сортируется по fitness.
- Возвращаются **топ-N лучших матриц** (обычно 20).
- Эти матрицы идут в **full evaluation** (строгий финальный тест).

---

## Производительность

### Временная сложность:
- Одно поколение: O(population_size × num_scenarios)
- Полная эволюция: O(population_size × num_generations × num_scenarios)
- Пример: 1000 × 500 × 2000 = 1 млрд вычислений (но оптимизировано!)

### Оптимизации:
- Векторизованная fitness-оценка (скорость ×100)
- Предварительное объединение сценариев (нет повторного стэкинга)
- Ранняя остановка по сходимости
- Малый набор сценариев в quick evaluation (2k вместо 10k)

### Память:
- Хранится только **текущая популяция**, нет истории.
- Сценарии создаются один раз и переиспользуются.
- Подходит даже для больших популяций.

---

**Итог:**  
`run_genetic_algorithm()` — это «двигатель» оптимизации, который:
- создаёт стартовую популяцию,
- эволюционирует матрицы,
- гарантирует соблюдение всех ограничений,
- отслеживает прогресс,
- завершает работу при сходимости,
- возвращает лучшие решения для финальной проверки.

In [ ]:
# ======================
# GENETIC ALGORITHM
# ======================

def run_genetic_algorithm(df: pd.DataFrame, merit_pool: float) -> List[MatrixCandidate]:
    """Main GA loop"""
    rng = np.random.default_rng(SEED_GA)
    
    print(f"\n{'='*80}")
    print("GENETIC ALGORITHM EVOLUTION")
    print(f"{'='*80}")
    print(f"\nMatrix: {NUM_CR_BINS} CR bins × {NUM_RATINGS} ratings")
    print("Orientation: Rows = CR bins (0=lowest CR), Cols = ratings (1..N)")
    
    # Runtime safeguard for large runs
    total_evals = GA_CONFIG['population_size'] * GA_CONFIG['num_generations']
    if len(df) > 5000 and total_evals > 60_000:
        print(f"\nLarge run detected ({len(df)} employees, {total_evals:,} evaluations).")
        print("Consider reducing population_size or num_generations for faster runtime.")
    
    print(f"\nGenerating {EVAL_CONFIG['quick_eval_scenarios']} evaluation scenarios...")
    quick_scenarios = generate_rating_scenarios(df, EVAL_CONFIG['quick_eval_scenarios'], True)
    S_quick = np.stack(quick_scenarios)
    print(f"Scenarios ready")
    
    zero_mask = compute_zero_support_mask(quick_scenarios)
    combined_zero_mask = zero_mask | STRICT_ZERO_MASK
    
    # Get anchor cells for reporting
    anchors = get_anchor_cells()
    anchor_positions = {(i, j) for _, (i, j, _) in anchors.items()}
    
    # Warn if anchors overlap with zero-support cells
    for anchor_type, (i, j, value) in anchors.items():
        if zero_mask[i, j]:
            print(f"Warning: Anchor cell at CR bin {i}, Rating {j+1} has zero payroll in all scenarios")
            print(f"    Pinning to {value*100:.2f}% will not affect budget since no employees in this cell")
    
    num_frozen = combined_zero_mask.sum()
    num_anchored = len(anchor_positions)
    total_fixed = num_frozen + num_anchored
    
    if total_fixed > 0:
        print(f"Fixed cells in matrix:")
        if num_frozen > 0:
            print(f"  - Zero cells: {num_frozen} (zero payroll: {zero_mask.sum()}, strict: {STRICT_ZERO_MASK.sum()})")
        if num_anchored > 0:
            print(f"  - Anchor cells: {num_anchored}")
            for anchor_type, (i, j, value) in anchors.items():
                print(f"    • CR bin {i}, Rating {j+1} = {value*100:.2f}%")
    
    print(f"\nInitializing population of {GA_CONFIG['population_size']} matrices...")
    population_matrices = initialize_population(GA_CONFIG['population_size'], rng)
    
    # Apply zero mask to all matrices and re-enforce all fixed cells
    for idx, mat in enumerate(population_matrices):
        mat[combined_zero_mask] = 0.0
        population_matrices[idx] = enforce_all_fixed_cells(mat) 
    
    population = []
    for mat in population_matrices:
        fitness, budget_sc, constraint_sc, success_rate, mean_dev = evaluate_fitness(
            mat, S_quick, merit_pool, BUDGET_TOLERANCE_LOWER, BUDGET_TOLERANCE_UPPER
        )
        population.append(MatrixCandidate(
            matrix=mat, fitness=fitness, budget_score=budget_sc,
            constraint_score=constraint_sc, success_rate=success_rate,
            mean_deviation=mean_dev, generation=0
        ))
    
    print(f"Initial population evaluated")
    print(f"  Best fitness: {max(p.fitness for p in population):.6f}")
    
    # Check for low initial success rate
    best_initial = max(population, key=lambda x: x.fitness)
    if best_initial.success_rate < 0.3:
        print(f"\nWarning: Low initial success rate ({best_initial.success_rate:.1%}).")
        print("Consider: (1) relaxing budget tolerances, or")
        print("          (2) increasing SCENARIO_CONCENTRATION to reduce variance.")
    
    best_fitness_history = []
    generations_without_improvement = 0
    
    for gen in range(GA_CONFIG['num_generations']):
        population.sort(key=lambda x: x.fitness, reverse=True)
        
        best_fitness = population[0].fitness
        best_fitness_history.append(best_fitness)
        
        if len(best_fitness_history) > 1:
            improvement = best_fitness - best_fitness_history[-2]
            if improvement < GA_CONFIG['convergence_threshold']:
                generations_without_improvement += 1
            else:
                generations_without_improvement = 0
        
        if generations_without_improvement >= GA_CONFIG['convergence_patience']:
            print(f"\nConverged at generation {gen+1}")
            break
        
        if (gen + 1) % 10 == 0 or gen == 0:
            print(f"\nGeneration {gen+1}/{GA_CONFIG['num_generations']}")
            print(f"  Best fitness: {best_fitness:.6f}")
            print(f"  Success rate: {population[0].success_rate:.1%}")
            print(f"  Mean deviation: {population[0].mean_deviation:.4f}")
        
        next_generation = []
        
        elite = population[:GA_CONFIG['elite_size']]
        next_generation.extend([MatrixCandidate(
            matrix=e.matrix.copy(), fitness=e.fitness, budget_score=e.budget_score,
            constraint_score=e.constraint_score, success_rate=e.success_rate,
            mean_deviation=e.mean_deviation, generation=gen+1
        ) for e in elite])
        
        while len(next_generation) < GA_CONFIG['population_size']:
            parent1 = tournament_selection(population, GA_CONFIG['tournament_size'], rng)
            parent2 = tournament_selection(population, GA_CONFIG['tournament_size'], rng)
            
            if rng.random() < GA_CONFIG['crossover_rate']:
                child_matrix = crossover(parent1.matrix, parent2.matrix, rng)
            else:
                child_matrix = parent1.matrix.copy()
            
            child_matrix = mutate(child_matrix, GA_CONFIG['mutation_rate'], rng)
            
            if not np.isfinite(child_matrix).all():
                child_matrix = repair_matrix(np.nan_to_num(child_matrix, nan=CONSTRAINTS['cell_min']))
            
            # Apply zero mask and anchor cells
            child_matrix[combined_zero_mask] = 0.0
            child_matrix = enforce_all_fixed_cells(child_matrix)
            
            fitness, budget_sc, constraint_sc, success_rate, mean_dev = evaluate_fitness(
                child_matrix, S_quick, merit_pool, BUDGET_TOLERANCE_LOWER, BUDGET_TOLERANCE_UPPER
            )
            
            next_generation.append(MatrixCandidate(
                matrix=child_matrix, fitness=fitness, budget_score=budget_sc,
                constraint_score=constraint_sc, success_rate=success_rate,
                mean_deviation=mean_dev, generation=gen+1
            ))
        
        population = next_generation
    
    population.sort(key=lambda x: x.fitness, reverse=True)
    
    print(f"\n{'='*80}")
    print("EVOLUTION COMPLETE")
    print(f"{'='*80}")
    print(f"Final best fitness: {population[0].fitness:.6f}")
    
    return population[:EVAL_CONFIG['num_top_candidates']]

## Финальная оценка (Full Evaluation) и удаление дубликатов (Deduplication)

### Зачем нужен этап deduplication?

Генетический алгоритм нередко приходит к очень похожим или даже одинаковым матрицам.  
Если их не удалить:
- Будет тратиться время на повторную оценку одинаковых решений,
- В итоговом рейтинге одно и то же решение может появиться несколько раз.

### deduplicate_candidates()

**Что делает:**
- Сравнивает каждую матрицу с уже встреченными,
- Считает дубликатом, если значения совпадают почти полностью (порог очень строгий: 1e-6),
- Удаляет дубли, оставляя только уникальные матрицы.

**Когда применяется:**
- Перед финальной (дорогой) оценкой,
- Уменьшает количество матриц на 20–50%,
- Снижает вычислительную нагрузку.

---

## Полная оценка (Full Evaluation)

### Зачем она нужна?

Во время эволюции используется **quick evaluation** (например, 2000 сценариев), чтобы работать быстро.  
Но финальные решения должны быть проверены **точно** → используется **full evaluation** (например, 10 000 сценариев).

**Компромисс:**
- Быстрота во время обучения,
- Максимальная точность при выборе лучших решений.

---

## Этапы full_evaluation()

### 1. Генерация полного набора сценариев
Создаётся большой набор (например, 10 000 сценариев), включая стресс-сценарии.

### 2. Обновление zero mask
На большем количестве сценариев некоторые ячейки могут стать постоянно нулевыми.  
Создаётся новый `combined_zero_mask`, более точный и устойчивый.

### 3. Локальная донастройка (опционально)
Применяется **только к топ-5 матрицам**, потому что:
- Локальная оптимизация дорогая,
- Лучше дорабатывать сильные решения,
- Нет смысла тратить ресурсы на слабые,
- Избегается переобучение.

Локальная донастройка использует небольшой поднабор сценариев (например, 100), чтобы ускорить расчёт.

### 4. Полная переоценка
Каждая матрица (после донастройки или как есть) заново вычисляет:
- fitness,
- budget_score,
- constraint_score,
- success_rate,
- mean_deviation.

Оценка проводится **по всем 10 000 сценариям**.

### 5. Финальный рейтинг
Матрицы сортируются по fitness (от лучшей к худшей).  
Результат — окончательный список лучших решений, готовых к экспорту и использованию.

---

## Почему quick eval и full eval могут отличаться?

**Обычно:**
- Топ-3 остаются топ-3,
- Success rate может отличаться на 5–10%,
- Mean deviation обычно стабильнее.

**Если порядок сильно изменился:**
- Значит, матрица была «заточена» под quick-сценарии,
- Full evaluation показывает «истинное» качество,
- Это оправдывает необходимость финального этапа.

---

## Итого

- **Deduplication** удаляет повторяющиеся решения → экономия времени.
- **Full Evaluation** даёт точную картину производительности.
- **Локальная донастройка** улучшает только лучшие решения, без перерасхода ресурсов.
- **Финальный рейтинг** обеспечивает надёжный выбор матриц для бизнеса.

---

## Генерация политики (Policy Generation)

### Зачем?

Даже если матрица повышений идеальна, итоговый бюджет зависит от того, **как менеджеры распределят рейтинги**.  
HR и менеджерам нужна **чёткая инструкция**: какие доли рейтингов безопасны для бюджета.

### generate_policy_for_matrix()

**Как работает:**

#### Шаг 1. Монте-Карло семплирование
- Генерируется ~200 возможных распределений рейтингов,
- Для каждого считается бюджет с текущей матрицей,
- Оставляются **только те**, где бюджет в пределах допустимого диапазона.

→ Получаем набор **успешных** распределений.

#### Шаг 2. Извлечение границ для каждого рейтинга
Для каждого уровня рейтинга:
- Берутся все успешные проценты,
- Вычисляются:
  - **Target %** — целевое значение (из MATR),
  - **Hard Min % (P10)** — нижняя надёжная граница,
  - **Hard Max % (P90)** — верхняя надёжная граница.

Почему P10/P90?
- Это устойчиво к выбросам,
- 80% «безопасной зоны»,
- Лучше, чем min/max (слишком экстремально).

---

## Как читать политику

Пример:
- Rating 3: target=50%, hard_min=45%, hard_max=55%

Значит:
- Цель — 50%
- Можно 45–55% без риска
- Выше или ниже — вероятность выхода из бюджета повышается

---

## Бизнес-применение

**Калибровка менеджеров**  
«У тебя Rating 5 = 25%, но максимум безопасный = 13% → бюджет под угрозой.»

**Политики HR**  
«Рейтинги должны распределяться в указанных диапазонах.»

**Сценарное планирование**  
«Что если все будут ставить высокие рейтинги?»

---

## Если viable-политик нет

Это редкость, но возможно:
- Бюджет слишком строгий,
- Матрица слишком агрессивная.

В этом случае:
- Возвращается пустой список,
- Пользователю выводится предупреждение,
- Рекомендуется ослабить бюджет или уменьшить вариативность сценариев.

---

**Итог:**  
Этот этап превращает техническую матрицу в **практические бизнес-правила**, которые помогает менеджерам удерживать бюджет под контролем.


In [ ]:
# ======================
# FINAL EVALUATION
# ======================

def deduplicate_candidates(candidates: List[MatrixCandidate], tolerance: float = 1e-6) -> List[MatrixCandidate]:
    """Remove duplicate matrices, keeping the best fitness for each unique matrix"""
    if not candidates:
        return []
    
    unique_candidates = []
    seen_matrices = []
    
    for candidate in candidates:
        is_duplicate = False
        for seen_matrix in seen_matrices:
            # Check if matrices are identical within tolerance
            if np.allclose(candidate.matrix, seen_matrix, atol=tolerance):
                is_duplicate = True
                break
        
        if not is_duplicate:
            unique_candidates.append(candidate)
            seen_matrices.append(candidate.matrix.copy())
    
    return unique_candidates


def full_evaluation(candidates: List[MatrixCandidate], df: pd.DataFrame, merit_pool: float) -> List[MatrixCandidate]:
    print(f"\n{'='*80}")
    print("FULL EVALUATION")
    print(f"{'='*80}")

    print(f"\nGenerating {EVAL_CONFIG['full_eval_scenarios']} scenarios...")
    full_scenarios = generate_rating_scenarios(df, EVAL_CONFIG['full_eval_scenarios'], True)
    S_full = np.stack(full_scenarios)
    zero_mask_full = compute_zero_support_mask(full_scenarios)
    combined_zero_mask = zero_mask_full | STRICT_ZERO_MASK
    print(f"Full scenario set ready")

    refined_candidates = []
    for i, candidate in enumerate(candidates):
        mat = candidate.matrix.copy()
        mat[combined_zero_mask] = 0.0
        mat = enforce_all_fixed_cells(mat)

        if i < 5:
            refined_matrix = local_refinement(
                mat, S_full[:100], merit_pool, BUDGET_TOLERANCE_LOWER, BUDGET_TOLERANCE_UPPER
            )
            refined_matrix[combined_zero_mask] = 0.0
            refined_matrix = enforce_all_fixed_cells(refined_matrix)
        else:
            refined_matrix = mat

        fitness, budget_sc, constraint_sc, success_rate, mean_dev = evaluate_fitness(
            refined_matrix, S_full, merit_pool, BUDGET_TOLERANCE_LOWER, BUDGET_TOLERANCE_UPPER
        )

        refined_candidates.append(MatrixCandidate(
            matrix=refined_matrix, fitness=fitness, budget_score=budget_sc,
            constraint_score=constraint_sc, success_rate=success_rate,
            mean_deviation=mean_dev, generation=candidate.generation
        ))

        if (i + 1) % 5 == 0:
            print(f"  Evaluated {i+1}/{len(candidates)}")

    refined_candidates.sort(key=lambda x: x.fitness, reverse=True)
    print(f"Full evaluation complete")
    return refined_candidates


def generate_policy_for_matrix(matrix: np.ndarray, df: pd.DataFrame, 
                              merit_pool: float) -> List[PolicyGuidance]:
    """Generate management policy for a specific matrix"""
    rng = np.random.default_rng()
    successful_dists = []
    
    base_probs = np.array([TARGET_RATING_DISTRIBUTION[r] for r in sorted(RATING_ORDER)])
    
    for _ in range(200):
        perturbed = dirichlet_sample_from_target(base_probs, SCENARIO_CONCENTRATION, rng)
        dist = {r: p for r, p in zip(sorted(RATING_ORDER), perturbed)}
        
        S = create_salary_matrix(df, dist, rng)
        cost = (S * matrix).sum()
        deviation = (cost - merit_pool) / merit_pool
        
        # Asymmetric budget check
        if deviation >= -BUDGET_TOLERANCE_LOWER and deviation <= BUDGET_TOLERANCE_UPPER:
            successful_dists.append({r: dist[r] * 100 for r in RATING_ORDER})
    
    if not successful_dists:
        return []
    
    policies = []
    for rating in sorted(RATING_ORDER):
        values = [d[rating] for d in successful_dists]
        policies.append(PolicyGuidance(
            rating=rating,
            target_pct=TARGET_RATING_DISTRIBUTION[rating] * 100,
            hard_min_pct=np.percentile(values, 10),
            hard_max_pct=np.percentile(values, 90)
        ))
    
    return policies

## Reporting & Export

### **1. Summary Printing (print_summary)**

Выводит компактную таблицу топ-кандидатов:

* Rank (позиция)
* Fitness (общий балл)
* Success% (доля сценариев в бюджете)
* Budget Score (качество попадания в бюджет)
* Constraint Score (качество соблюдения ограничений)

Показывает до 20 лучших решений.
**Цель:** быстро понять, какие матрицы лидируют.

---

### **2. Экспорт в Excel (export_results)**

Создаётся **один Excel-файл** с несколькими листами (1 лист = 1 матрица).

#### **Структура листа:**

**🟦 METADATA (первые строки)**
Содержит ключевые метрики матрицы:

* Fitness
* Success Rate
* Budget Score
* Constraint Score
* Differentiation Score (видимость различий между рейтингами)
* Mean Deviation (среднее отклонение бюджета)

**🟦 MERIT MATRIX (%)**

* Строки = CR bins
* Столбцы = Ratings
* Значения = % повышения (0–100%)
* Все округлено до 2 знаков

CR bins подписаны:

* Если custom – показываются диапазоны (например: `[0.90, 1.10)`).
* Если auto – добавляется описание (например: `Lowest CR - Best Positioned`).

**🟦 POLICY GUIDANCE**
Для каждого рейтинга:

* Target %
* Hard Min %
* Hard Max %

Если невозможно построить политику — выводится сообщение.

---

### **3. Дополнительные файлы**

#### ✅ Summary CSV

CSV-файл с ключевыми метриками:

* rank, fitness, success_rate, budget_score, constraint_score, mean_deviation

**Цель:** быстро сравнить решения, удобно для анализа в pandas/Excel.

#### ✅ Configuration JSON

Полный «паспорт» оптимизации:

* Настройки рейтингов и CR-бинов
* Источник целевого распределения
* Budget tolerance
* Monte Carlo параметры
* Весовые коэффициенты фитнеса
* Информация о фиксированных ячейках
* Лучшая fitness и differentiation
* Количество уникальных кандидатов

**Цель:** 100% воспроизводимость и прозрачность.

---

### **4. Детальный анализ матрицы (analyze_matrix_application)**

Применяет лучшую матрицу к реальной популяции сотрудников.

**Что анализируется:**

**✅ Budget Summary**

* Общая зарплата
* Целевой бюджет и фактический
* Отклонение
* В пределах бюджета? (YES/NO)

**✅ Merit by Rating**
Показывает средний % повышения и количество сотрудников по каждому рейтингу.

**✅ Zero Merit**
Сколько сотрудников получат 0% — важно для HR и коммуникаций.

---

### **5. Анализ риска (analyze_scenario_risk)**

Проверяет матрицу на 1000+ сценариев (включая стрессовые).

**Выводит:**

* Диапазон допуска (budget tolerance)
* Success Rate (% сценариев в бюджете)
* Mean Deviation (среднее отклонение)
* Std Dev (разброс затрат)

**Интерпретация:**

* > 80% → низкий риск ✅
* 60–80% → средний риск ⚠️
* <60% → высокий риск ❌

**Цель:** понять реальную устойчивость матрицы.

---

In [ ]:
# ======================
# REPORTING
# ======================

def print_summary(candidates: List[MatrixCandidate]):
    """Print summary of top candidates"""
    print(f"\n{'='*80}")
    print("TOP CANDIDATE MATRICES")
    print(f"{'='*80}\n")
    
    print(f"{'Rank':<6} {'Fitness':<10} {'Success%':<10} {'Budget':<10} {'Constraint':<12}")
    print("-" * 60)
    
    for i, cand in enumerate(candidates[:20], 1):
        print(f"{i:<6} {cand.fitness:<10.6f} {cand.success_rate*100:<10.1f} "
              f"{cand.budget_score:<10.6f} {cand.constraint_score:<12.6f}")


def export_results(candidates: List[MatrixCandidate], df: pd.DataFrame, merit_pool: float):
    """Export all results to a single Excel workbook with multiple sheets"""
    
    if not candidates:
        print("No candidates to export")
        return
    
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    print(f"\n{'='*80}")
    print("EXPORTING RESULTS")
    print(f"{'='*80}")
    
    I, J = candidates[0].matrix.shape
    rating_cols = [f"Rating {RATING_ORDER[j]}" for j in range(J)]
    
    # Create descriptive CR bin labels - show ranges when custom bins are used
    cr_labels = []
    if CUSTOM_CR_BINS is not None:
        for i in range(I):
            lo, hi = CR_BIN_EDGES[i], CR_BIN_EDGES[i+1]
            if np.isinf(hi):
                cr_labels.append(f"CR Bin {i} [{lo:.2f}, ∞)")
            else:
                cr_labels.append(f"CR Bin {i} [{lo:.2f}, {hi:.2f})")
    else:
        for i in range(I):
            if i == 0:
                cr_labels.append(f"CR Bin {i} (Lowest CR - Best Positioned)")
            elif i == I - 1:
                cr_labels.append(f"CR Bin {i} (Highest CR - Worst Positioned)")
            else:
                cr_labels.append(f"CR Bin {i}")
    
    # Create single Excel workbook
    excel_path = os.path.join(OUTPUT_DIR, "merit_matrices_all_scenarios.xlsx")
    
    print(f"\nCreating Excel workbook with {len(candidates)} scenarios...")
    
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        for i, cand in enumerate(candidates, 1):
            sheet_name = f"Rank_{i:02d}"
            
            # Convert to percentage and round to 2 decimals
            matrix_pct = np.round(cand.matrix * 100, 2)
            
            # Create matrix dataframe
            df_matrix = pd.DataFrame(
                matrix_pct,
                index=cr_labels,
                columns=rating_cols
            )
            
            # Add metadata rows - include differentiation score
            diff_sc = differentiation_score(cand.matrix)
            metadata = pd.DataFrame({
                'Metric': ['Fitness', 'Success Rate (%)', 'Budget Score', 'Constraint Score', 'Diff Score', 'Mean Deviation (%)'],
                'Value': [
                    f"{cand.fitness:.6f}",
                    f"{cand.success_rate*100:.2f}",
                    f"{cand.budget_score:.6f}",
                    f"{cand.constraint_score:.6f}",
                    f"{diff_sc:.6f}",
                    f"{cand.mean_deviation*100:.4f}"
                ]
            })
            
            # Generate policy
            policy = generate_policy_for_matrix(cand.matrix, df, merit_pool)
            
            if policy:
                policy_df = pd.DataFrame([{
                    'Rating': p.rating,
                    'Target %': f"{p.target_pct:.2f}",
                    'Hard Min %': f"{p.hard_min_pct:.2f}",
                    'Hard Max %': f"{p.hard_max_pct:.2f}"
                } for p in policy])
            else:
                policy_df = pd.DataFrame({'Note': ['No successful policy distributions found']})
            
            # Write to sheet with sections
            # Section 1: Metadata
            metadata.to_excel(writer, sheet_name=sheet_name, index=False, startrow=0)
            
            # Section 2: Merit Matrix (with blank row separator)
            startrow_matrix = len(metadata) + 2
            writer.sheets[sheet_name].cell(row=startrow_matrix, column=1, value="MERIT MATRIX (%)")
            df_matrix.to_excel(writer, sheet_name=sheet_name, startrow=startrow_matrix + 1)
            
            # Section 3: Policy Guidance (with blank row separator)
            startrow_policy = startrow_matrix + len(df_matrix) + 4
            writer.sheets[sheet_name].cell(row=startrow_policy, column=1, value="POLICY GUIDANCE")
            policy_df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=startrow_policy + 1)
            
            if (i) % 5 == 0:
                print(f"  Exported {i}/{len(candidates)} scenarios")
    
    print(f"\nExported to single Excel file: '{excel_path}'")
    print(f"  - {len(candidates)} sheets (one per scenario)")
    print(f"  - Each sheet contains: metadata, merit matrix, and policy guidance")
    
    # Also export summary CSV
    summary = pd.DataFrame([{
        'rank': i,
        'fitness': c.fitness,
        'success_rate': c.success_rate,
        'budget_score': c.budget_score,
        'constraint_score': c.constraint_score,
        'mean_deviation': c.mean_deviation
    } for i, c in enumerate(candidates, 1)])
    summary_path = os.path.join(OUTPUT_DIR, "candidate_summary.csv")
    summary.to_csv(summary_path, index=False)
    print(f"  - Summary CSV: '{summary_path}'")
    
    # Export configuration - include Monte Carlo settings
    config = {
        'num_ratings': NUM_RATINGS,
        'rating_order': RATING_ORDER,
        'target_distribution': {str(k): v for k, v in TARGET_RATING_DISTRIBUTION.items()},
        'custom_target_distribution_used': CUSTOM_TARGET_DISTRIBUTION is not None,
        'num_cr_bins': NUM_CR_BINS,
        'cr_bin_edges': [float(e) if not np.isinf(e) else 'inf' for e in CR_BIN_EDGES],
        'custom_cr_bins_used': CUSTOM_CR_BINS is not None,
        'cr_bin_interpretation': 'Bin 0 = Lowest CR (best positioned), Higher bins = Higher CR (worse positioned)',
        'merit_pool_value': MERIT_POOL_VALUE,
        'budget_tolerance_lower': BUDGET_TOLERANCE_LOWER,
        'budget_tolerance_upper': BUDGET_TOLERANCE_UPPER,
        'monte_carlo': {
            'concentration': SCENARIO_CONCENTRATION,
            'hard_zero_ratings': HARD_ZERO_RATINGS,
            'stress_honors_zeros': HARD_ZERO_RATINGS,
            'note': 'Higher concentration = less variance around target distribution'
        },
        'fitness_weights': FITNESS_WEIGHTS,
        'diff_weight': DIFF_WEIGHT,
        'best_diff_score': float(differentiation_score(candidates[0].matrix)),
        'strict_zero_cells': {str(k): v for k, v in (FORCE_ZERO_CELLS or {}).items()} if FORCE_ZERO_CELLS else None,
        'anchor_cell_max': ANCHOR_CELL_MAX,
        'anchor_cell_min': ANCHOR_CELL_MIN,
        'best_fitness': candidates[0].fitness,
        'num_unique_candidates': len(candidates)
    }
    
    config_path = os.path.join(OUTPUT_DIR, "optimization_config.json")
    with open(config_path, "w") as f:
        json.dump(config, f, indent=2)
    print(f"  - Configuration JSON: '{config_path}'")


def analyze_matrix_application(matrix: np.ndarray, df: pd.DataFrame, merit_pool: float):
    """Comprehensive analysis"""
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    print(f"\n{'='*80}")
    print("DETAILED ANALYSIS")
    print(f"{'='*80}")
    
    df_analysis = df.copy()
    rng = np.random.default_rng(SEED_BASE_POPULATION)
    df_analysis['rating'] = rng.choice(
        RATING_ORDER,
        size=len(df_analysis),
        p=[TARGET_RATING_DISTRIBUTION[r] for r in sorted(RATING_ORDER)]
    )
    rating_to_idx = {r: i for i, r in enumerate(sorted(RATING_ORDER))}
    df_analysis['rating_idx'] = df_analysis['rating'].map(rating_to_idx)
    
    df_analysis['merit_pct'] = df_analysis.apply(
        lambda row: matrix[int(row['cr_bin']), int(row['rating_idx'])], axis=1
    )
    df_analysis['merit_amount'] = df_analysis['base_salary'] * df_analysis['merit_pct']
    
    total_merit_cost = df_analysis['merit_amount'].sum()
    actual_merit_pct = total_merit_cost / df_analysis['base_salary'].sum()
    variance = (total_merit_cost - merit_pool) / merit_pool
    
    print(f"\nBUDGET SUMMARY")
    print(f"Total Payroll:    {df_analysis['base_salary'].sum():,.0f}")
    print(f"Target Merit:     {merit_pool:,.0f} ({MERIT_POOL_VALUE:.2%})")
    print(f"Actual Merit:     {total_merit_cost:,.0f} ({actual_merit_pct:.2%})")
    print(f"Variance:         {total_merit_cost - merit_pool:+,.0f} ({variance:+.2%})")
    within = (variance >= -BUDGET_TOLERANCE_LOWER) and (variance <= BUDGET_TOLERANCE_UPPER)
    print(f"Within Budget:    {'YES' if within else 'NO'}")
    
    print(f"\nMERIT BY RATING")
    for rating in sorted(RATING_ORDER):
        subset = df_analysis[df_analysis['rating'] == rating]
        if len(subset) > 0:
            avg_merit = subset['merit_pct'].mean() * 100
            count = len(subset)
            print(f"  Rating {rating}: {avg_merit:5.2f}% avg (n={count})")
    
    zero_merit = df_analysis[df_analysis['merit_pct'] == 0]
    if len(zero_merit) > 0:
        print(f"\nZERO MERIT: {len(zero_merit)} employees ({len(zero_merit)/len(df_analysis)*100:.1f}%)")


def analyze_scenario_risk(matrix: np.ndarray, df: pd.DataFrame, merit_pool: float) -> None:
    """Scenario risk analysis"""
    print(f"\n{'='*80}")
    print("SCENARIO RISK ANALYSIS")
    print(f"{'='*80}")
    
    scenarios = generate_rating_scenarios(df, 1000, True)
    
    costs = []
    for S in scenarios:
        cost = (S * matrix).sum()
        costs.append(cost)
    
    deviations = np.array([(c - merit_pool) / merit_pool * 100 for c in costs])
    
    # Asymmetric budget check
    within = sum(1 for d in deviations if d >= -BUDGET_TOLERANCE_LOWER * 100 and d <= BUDGET_TOLERANCE_UPPER * 100)
    
    print(f"\nBudget Tolerance: {-BUDGET_TOLERANCE_LOWER*100:.1f}% to +{BUDGET_TOLERANCE_UPPER*100:.1f}%")
    print(f"Success Rate: {within/len(scenarios)*100:.1f}% ({within}/{len(scenarios)} scenarios within band)")
    print(f"Mean Deviation: {deviations.mean():+.2f}%")
    print(f"Std Dev: {deviations.std():.2f}%")

## Main Execution Function (main)

Главная функция, управляющая всем процессом.

### **Основные этапы:**

### Phase 1: Печать конфигурации

Показываются все ключевые параметры:

* Кол-во рейтингов
* CR-бин структура
* Источник распределения (custom/auto)
* Merit pool
* Budget tolerance
* Monte Carlo concentration
  **Цель:** прозрачность и проверка перед запуском.

---

### Phase 2: Валидация

Проверяются настройки.
Если что-то некорректно — показывается ошибка и выполнение останавливается.

---

### Phase 3: Загрузка данных

* Если Excel есть — загружается.
* Если нет — генерируются синтетические данные.
* Считается общий payroll и merit pool в долларах.

---

### Phase 4: Генетический алгоритм (быстрая оптимизация)

Запускается GA с quick evaluation (2k сценариев).
Получаем top кандидатов.

Если кандидатов нет — завершаем процесс.

---

### Phase 5: Full Evaluation (точная оценка)

Топ кандидаты тестируются на 10k сценариях.
Применяется локальная донастройка (только для топ-5).
Формируется финальный рейтинг.

Если нет успешных — завершаем.

---

### Phase 6: Reporting & Export

* Печать summary
* Генерация Excel/CSV/JSON
* Детальный анализ лучшей матрицы
* Анализ риска

---

### Phase 7: Завершение

Печать: **"OPTIMIZATION COMPLETE"**

---

## **Обработка ошибок**

* Нет файла данных → синтетика
* Нет кандидатов → сообщение и выход
* Все обрабатывается корректно, исключения наружу не выбрасываются

---

## **Entry Point**

Используется стандартный Python-подход:
`if __name__ == "__main__": main()`

Позволяет:

* Запуск как скрипт
* Импорт как модуль без автозапуска

---

# Итоговый workflow:

1. Конфигурация → проверка → загрузка данных
2. GA (quick eval) → топ кандидаты
3. Full eval (точность) → финальный список
4. Local refinement (дополировка лучших)
5. Экспорт (Excel/CSV/JSON)
6. Аналитика (бюджет + риск)

---

## Ключевые метрики:

* Success Rate (% успешных сценариев)
* Fitness (общий балл)
* Mean Deviation (точность бюджета)
* Constraint Score (структурная корректность)
* Differentiation (видимость различий по рейтингам)

---

## Типичные результаты:

* Success Rate: **80–95%**
* Fitness: **0.90–0.98**
* Mean Deviation: **0.5–2%**
* Runtime: **5–30 минут**

In [1]:
# ======================
# MAIN
# ======================

def main():
    print("="*80)
    print("MERIT MATRIX OPTIMIZER v4.7")
    print("="*80)
    
    print(f"\n{'='*80}")
    print("CONFIGURATION")
    print(f"{'='*80}")
    print(f"Ratings: {NUM_RATINGS} (scale 1-{NUM_RATINGS})")
    
    # Display target distribution info
    if CUSTOM_TARGET_DISTRIBUTION is not None:
        if HARD_ZERO_RATINGS:
            print(f"Target Distribution Source: CUSTOM (exact with hard zeros), Monte Carlo perturbs around this mix.")
        else:
            print(f"Target Distribution Source: CUSTOM (exact), Monte Carlo perturbs around this mix.")
    else:
        print(f"Target Distribution Source: AUTO (bell curve), Monte Carlo perturbs around this.")
    
    print(f"CR Bins: {NUM_CR_BINS}")
    if CUSTOM_CR_BINS is not None:
        print(f"  Using CUSTOM CR bins:")
        for i in range(len(CR_BIN_EDGES) - 1):
            lower = CR_BIN_EDGES[i]
            upper = CR_BIN_EDGES[i+1]
            if np.isinf(upper):
                print(f"    Bin {i}: [{lower:.2f}, ∞)")
            else:
                print(f"    Bin {i}: [{lower:.2f}, {upper:.2f})")
    else:
        print(f"  Auto-generated: {CR_BIN_FIRST:.2f} to {CR_BIN_LAST:.2f}")
    
    print(f"Merit Pool: {MERIT_POOL_VALUE:.2%}")
    print(f"Budget Tolerance: -{BUDGET_TOLERANCE_LOWER:.1%} to +{BUDGET_TOLERANCE_UPPER:.1%}")
    print(f"Monte Carlo Concentration: {SCENARIO_CONCENTRATION:.2f} (higher = less variance)")
    
    print(f"\nTarget Distribution:")
    for rating, pct in sorted(TARGET_RATING_DISTRIBUTION.items()):
        print(f"  Rating {rating}: {pct*100:5.1f}%")
    
    validate_configuration()
    
    df = load_employee_data(DATA_FILE)
    
    base_payroll = df['base_salary'].sum()
    merit_pool = MERIT_POOL_VALUE * base_payroll
    
    print(f"\nLoaded {len(df)} employees")
    print(f"  Payroll: {base_payroll:,.0f}")
    print(f"  Merit Pool: {merit_pool:,.0f}")
    
    top_candidates = run_genetic_algorithm(df, merit_pool)
    
    if not top_candidates:
        print("\nNo candidates produced")
        return
    
    final_candidates = full_evaluation(top_candidates, df, merit_pool)
    
    if not final_candidates:
        print("\nNo candidates after evaluation")
        return
    
    print_summary(final_candidates)
    export_results(final_candidates, df, merit_pool)
    analyze_matrix_application(final_candidates[0].matrix, df, merit_pool)
    analyze_scenario_risk(final_candidates[0].matrix, df, merit_pool)
    
    print(f"\n{'='*80}")
    print("OPTIMIZATION COMPLETE")
    print(f"{'='*80}\n")


if __name__ == "__main__":
    main()

Using custom target distribution
Using custom CR bins: 5 bins defined

Constraint Analysis:
  Merit Pool: 13.00%
  Cell Range: 0.00% to 39.00% (span: 39.00%)
  Min Rating Step: 1.95% → Total: 7.80%
  Min CR Step: 0.97% → Total: 3.90%
  Feasibility Check: ✓ Good (20% utilization of available range)
MERIT MATRIX OPTIMIZER v4.7

CONFIGURATION
Ratings: 5 (scale 1-5)
Target Distribution Source: CUSTOM (exact), Monte Carlo perturbs around this mix.
CR Bins: 5
  Using CUSTOM CR bins:
    Bin 0: [0.00, 0.80)
    Bin 1: [0.80, 0.90)
    Bin 2: [0.90, 1.10)
    Bin 3: [1.10, 1.20)
    Bin 4: [1.20, ∞)
Merit Pool: 13.00%
Budget Tolerance: -5.0% to +5.0%
Monte Carlo Concentration: 166.67 (higher = less variance)

Target Distribution:
  Rating 1:  10.0%
  Rating 2:  15.0%
  Rating 3:  50.0%
  Rating 4:  15.0%
  Rating 5:  10.0%
Configuration validated
Loaded data from 'Misha.xlsx'

Loaded 25 employees
  Payroll: 50,257,241
  Merit Pool: 6,533,441

GENETIC ALGORITHM EVOLUTION

Matrix: 5 CR bins × 5 